In [4]:
import pandas as pd
import numpy as np
import geopandas as gp
import os
import maup
from op_verification.reference_data import *
from oh_disag_toolbox import *

pd.set_option("display.max_columns", None)

In [5]:
def modify_P0040001_P0050003(block_gdf):
    block_gdf['P0040001-P0050003'] = block_gdf['P0040001'] - block_gdf['P0050003']
    print("Error raised iff the number of incarcerated blocks with non-zero population does NOT equal the number of modified vap blocks.")
    assert (len(block_gdf[block_gdf['P0050003']!=0]) - len(block_gdf[block_gdf['P0040001']!=block_gdf['P0040001-P0050003']])) == 0
    block_gdf.loc[block_gdf['P0040001-P0050003']<0, 'P0040001-P0050003'] = 0

    return block_gdf


def gdf_to_df(gdf):
    df = gdf[gdf.columns[~gdf.columns.str.contains('geometry')]]
    return df


def prim_gen_join(prec_gen_gdf, df_prec_prim, DIST_ID):
    #**For n/a values in prim that have cd assignments in gen, apply the cd to the prim file as well**
    gen_id_to_dist_dict = pd.Series(prec_gen_gdf[DIST_ID].values, index = prec_gen_gdf['UNIQUE_ID']).to_dict()

    na_in_prim_not_in_gen = set(df_prec_prim['UNIQUE_ID'][df_prec_prim[DIST_ID]=='n/a'])-set(prec_gen_gdf['UNIQUE_ID'][prec_gen_gdf[DIST_ID]=='n/a'])
    df_prec_prim.loc[df_prec_prim['UNIQUE_ID'].isin(na_in_prim_not_in_gen), DIST_ID] = df_prec_prim['UNIQUE_ID'].map(gen_id_to_dist_dict)
    #**create join ids**
    df_prec_prim['prim_gen_join_id'] = df_prec_prim['GEOID20']+df_prec_prim[DIST_ID]
    prec_gen_gdf['prim_gen_join_id'] = prec_gen_gdf['GEOID20']+prec_gen_gdf[DIST_ID]
    
    #**Check that the GEOID20 + CD unique count is the same as the expected length of the df. It is.**
    assert (prec_gen_gdf['prim_gen_join_id']).nunique() == prec_gen_gdf['UNIQUE_ID'].nunique()
    assert (df_prec_prim['prim_gen_join_id']).nunique() == df_prec_prim['UNIQUE_ID'].nunique()
    #**Check that the length of unique vals in gen not in prim matches the number of additional rows gen has compared to prim.**
    assert len(set(prec_gen_gdf['prim_gen_join_id']) - set(df_prec_prim['prim_gen_join_id'])) == (prec_gen_gdf['UNIQUE_ID'].nunique() - df_prec_prim['UNIQUE_ID'].nunique())
    #**Check that all unique ids in prim are in gen df**
    assert (set(df_prec_prim['prim_gen_join_id']) - set(prec_gen_gdf['prim_gen_join_id'])) == set()
    
    #**Join prim and gen gdfs**
    prec_gdf = prec_gen_gdf.merge(df_prec_prim, how='outer', on=['prim_gen_join_id'])
    assert (prec_gdf['geometry'].isna().any() == False)
    assert (prec_gdf.shape[0] ==prec_gen_gdf.shape[0])

    return prec_gdf


def maup_pre_vote_setup(block_gdf, precinct_gdf, PRECID_block, DISTID_block, DISTID_prec):
    bgdf = maup_assignment_series(block_gdf, precinct_gdf)
    assert bgdf['maup_assignment'].isna().any() == False
    bgdf[PRECID_block] = bgdf['maup_assignment'].map(lambda idx: str(precinct_gdf.loc[idx]["UNIQUE_ID"]))
    bgdf[DISTID_block] = bgdf["maup_assignment"].map(lambda idx: str(precinct_gdf.loc[idx][DISTID_prec]))
    
    return bgdf


def assign_pop_to_zero_pop_prec(bgdf, PRECID, POP_COL, VAP_MOD):
    blk_groupedby_prec = bgdf.groupby(PRECID).sum()
    zero_pop_prec = blk_groupedby_prec.index[blk_groupedby_prec[POP_COL]==0]
    '''DECISION: Assign 1 to all blocks in precincts or only the precincts with votes?
       Consider: prec/blocks with no votes will not get any votes whether pop is zero or not, and easier to
                 assign all of them
       Conclude: for now assign 1 to all since outcome the same and easier
            '''
    bgdf[VAP_MOD] = bgdf[POP_COL]
    #assert that all in the zero pop list have zero pop, assert will return error if this is false
    assert (bgdf.loc[bgdf[PRECID].isin(zero_pop_prec), VAP_MOD] == 0).all()
    #Here assign that pop to 1 just so votes get distributed
    bgdf.loc[bgdf[PRECID].isin(zero_pop_prec), VAP_MOD] = 1
    return bgdf


def column_total_check(election_columns, block_gdf, precinct_gdf):
    for val in election_columns:
        vote_dif = block_gdf[val].sum()-precinct_gdf[val].sum()
        if (abs(vote_dif) <=1e-1):
            print(val+": EQUAL", ' - total: ', 'block:', str(block_gdf[val].sum()), 'prec:', str(precinct_gdf[val].sum()), 'diff:', block_gdf[val].sum()-precinct_gdf[val].sum())
        else:
            print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES", ' - block total: ', str(block_gdf[val].sum()), ', precinct total: ', str(precinct_gdf[val].sum()))     
            
            
def county_sum_check(prec_gdf, block_gdf, prec_county_col, blk_county_col):
    prec_gpbycty = prec_gdf.groupby([prec_county_col]).sum()
    blk_gpbycty = block_gdf.groupby([blk_county_col]).sum()
    cty_w_difs_list = []
    for i in range(0,prec_gdf[prec_county_col].nunique()):
        for col in election_cols(prec_gdf):
            county_diff = blk_gpbycty[col].iloc[i] - prec_gpbycty[col].iloc[i]
            if (abs(county_diff)>0.5):
                cty_w_difs_list.append(blk_gpbycty.index[i])
                print("county fips: ", blk_gpbycty.index[i], prec_gpbycty.index[i], col, "block: ", blk_gpbycty[col].iloc[i], 
                      "prec: ", prec_gpbycty[col].iloc[i], "diff: ", county_diff)
    print("counties to check: ", set(cty_w_difs_list))
    print("County check complete!")
    
    
def precinct_sum_check(prec_gdf, block_gdf, blk_prec_id):
    prec_gpbyprec = prec_gdf.groupby(['UNIQUE_ID']).sum()
    blk_gpbyprec = block_gdf.groupby([blk_prec_id]).sum()
    prec_w_difs_list = []
    for i in range(0,prec_gdf['UNIQUE_ID'].nunique()):
        for col in election_cols(prec_gdf):
            prec_diff = round(blk_gpbyprec[col].iloc[i], 1) - round(prec_gpbyprec[col].iloc[i], 1)
            if (prec_diff!=0):
                prec_w_difs_list.append(prec_gpbycty.index[i])
                print("prec id: ", blk_gpbyprec.index[i], prec_gpbyprec.index[i], col, "block: ", round(blk_gpbyprec[col].iloc[i],1), 
                      "prec: ", round(prec_gpbyprec[col].iloc[i],1), "diff: ", prec_diff)
    print("precs to check: ", set(prec_w_difs_list))
    print("Precinct check complete!")
    
    
def check_block_to_prec_pop_prop_sum(pop_col_list, VAP_MOD_COL, PREC_COL):
    pop_col_list = pop_col_list + ['GEOID20', 'P0040001', 'P0050003']
    popcd = b_merge[pop_col_list]
    prec_popcd = popcd.groupby(PREC_COL).sum().reset_index()

    prec_to_pop_dict_gcd = pd.Series(prec_popcd[VAP_MOD_COL].values, index = prec_popcd[PREC_COL]).to_dict()

    popcd['prec_popgcd'] = popcd[PREC_COL].map(prec_to_pop_dict_gcd)
    popcd['propgcd'] = popcd[VAP_MOD_COL]/popcd['prec_popgcd']

    print(popcd.groupby(PREC_COL).sum()['propgcd'].sum())
    print(popcd[PREC_COL].nunique())

# Disaggregation

## Load raw data

**Load in block file and create preliminary modified block column**

In [6]:
block_gdf = Imports('oh', 'b').import_block_file()
block_gdf = modify_P0040001_P0050003(block_gdf)

File imports initiated
block shape:  (276428, 14)
Error raised iff the number of incarcerated blocks with non-zero population does NOT equal the number of modified vap blocks.


In [7]:
neg_pop_blocks = list(block_gdf['GEOID20'][block_gdf['P0040001-P0050003']<0])

In [8]:
neg_pop_blocks

[]

**Load in precinct files**

In [9]:
prec_prim_st_gdf = Imports('oh', 'st').import_prec_prim_file()
prec_gen_st_gdf = Imports('oh', 'st').import_prec_gen_file()
prec_prim_cong_gdf = Imports('oh', 'cong').import_prec_prim_file()
prec_gen_cong_gdf = Imports('oh', 'cong').import_prec_gen_file()
prec_prim_sldl_gdf = Imports('oh', 'sldl').import_prec_prim_file()
prec_gen_sldl_gdf = Imports('oh', 'sldl').import_prec_gen_file()
prec_prim_sldu_gdf = Imports('oh', 'sldu').import_prec_prim_file()
prec_gen_sldu_gdf = Imports('oh', 'sldu').import_prec_gen_file()

File imports initiated
prec_prim shape:  (8941, 25)
File imports initiated
prec_gen shape:  (8941, 18)
File imports initiated
prec_prim shape:  (9067, 80)
File imports initiated
prec_gen shape:  (9089, 46)
File imports initiated
prec_prim shape:  (9033, 221)
File imports initiated
prec_gen shape:  (9067, 187)
File imports initiated
prec_prim shape:  (8941, 57)
File imports initiated
prec_gen shape:  (8941, 42)


## Precinct file join + maup

### State testing

**Do the state files match? If so why did I run separately below?
Not thinking I guess - can run together, the precincts match!
Shall I assign the countyfp from this file? If we are assignining all other boundaries than likely makes sense**

In [10]:
set(prec_prim_st_gdf.UNIQUE_ID) - set(prec_gen_st_gdf.UNIQUE_ID)

set()

In [11]:
set(prec_gen_st_gdf.UNIQUE_ID) - set(prec_prim_st_gdf.UNIQUE_ID)

set()

In [12]:
prec_st_gdf = prec_gen_st_gdf.merge(prec_prim_st_gdf, how='outer', on=['UNIQUE_ID', 'geometry'])

#### maup

In [13]:
bst = maup_pre_vote_setup(block_gdf, prec_st_gdf, 'PREC_ST', "ST_CTY", "COUNTYFP_x")

bst_mod = assign_pop_to_zero_pop_prec(bst, 'PREC_ST', 'P0040001-P0050003', 'VAP_MOD_ST')

bst_votes = assign_votes(
    'VAP_MOD_ST', election_cols(prec_st_gdf), prec_st_gdf, bst_mod
)

column_total_check(election_cols(prec_st_gdf), bst_votes, prec_st_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (8941, 41)
0  null assignments in maup series
VAP_MOD_ST  added to precinct_gdf based on maup assignment
G20PRERTRU: EQUAL  - total:  block: 3154833.999999999 prec: 3154834 diff: -9.313225746154785e-10
G20PREDBID: EQUAL  - total:  block: 2679165.0 prec: 2679165 diff: 0.0
G20PRELJOR: EQUAL  - total:  block: 67569.0 prec: 67569 diff: 0.0
G20PREGHAW: EQUAL  - total:  block: 18812.0 prec: 18812 diff: 0.0
GSSCKRKEN: EQUAL  - total:  block: 2735041.0 prec: 2735041 diff: 0.0
GSSCKDODO: EQUAL  - total:  block: 2231724.0 prec: 2231724 diff: 0.0
GSSCFDBRU: EQUAL  - total:  block: 2695072.0000000005 prec: 2695072 diff: 4.656612873077393e-10
GSSCFRFRE: EQUAL  - total:  block: 2174819.9999999995 prec: 2174820 diff: -4.656612873077393e-10
P20PREDBEN: EQUAL  - total:  block: 2030.0 prec: 2030 diff: 0.0
P20PREDBID: EQUAL  - total:  block: 647284.0 prec: 647284 diff: 0.0
P20PREDBLO: EQUAL  - total:  block: 28704.0 prec: 28704 diff: 0.0
P20PREDBUT: E

In [14]:
county_sum_check(prec_st_gdf, bst_votes, 'COUNTYFP_x', 'COUNTYFP20')

counties to check:  set()
County check complete!


### Congressional testing

In [15]:
df_prec_prim_cong = gdf_to_df(prec_prim_cong_gdf)
prec_cong_gdf = prim_gen_join(prec_gen_cong_gdf, df_prec_prim_cong, "CONG_DIST")

prec_cong_gdf['UNIQUE_ID'] = prec_cong_gdf['UNIQUE_ID_x']

#### maup

In [16]:
bcd = maup_pre_vote_setup(block_gdf, prec_cong_gdf, 'PREC_CD', "CONG_DIST", "CONG_DIST_x")

bcd_mod = assign_pop_to_zero_pop_prec(bcd, 'PREC_CD', 'P0040001-P0050003', 'VAP_MOD_CD')

bcd_votes = assign_votes(
    'VAP_MOD_CD', election_cols(prec_cong_gdf), prec_cong_gdf, bcd_mod
)

column_total_check(election_cols(prec_cong_gdf), bcd_votes, prec_cong_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (9089, 127)
0  null assignments in maup series
VAP_MOD_CD  added to precinct_gdf based on maup assignment
GCON01RCHA: EQUAL  - total:  block: 199559.9999999999 prec: 199560 diff: -8.731149137020111e-11
GCON01LKAH: EQUAL  - total:  block: 13691.999999999998 prec: 13692 diff: -1.8189894035458565e-12
GCON01DSCH: EQUAL  - total:  block: 172022.00000000006 prec: 172022 diff: 5.820766091346741e-11
GCON02DCAS: EQUAL  - total:  block: 146781.0 prec: 146781 diff: 0.0
GCON02RWEN: EQUAL  - total:  block: 230430.00000000003 prec: 230430 diff: 2.9103830456733704e-11
GCON03DBEA: EQUAL  - total:  block: 227420.00000000003 prec: 227420 diff: 2.9103830456733704e-11
GCON03RRIC: EQUAL  - total:  block: 93568.99999999999 prec: 93569 diff: -1.4551915228366852e-11
GCON04DFRE: EQUAL  - total:  block: 101897.0 prec: 101897 diff: 0.0
GCON04RJOR: EQUAL  - total:  block: 235875.00000000003 prec: 235875 diff: 2.9103830456733704e-11
GCON04LPER: EQUAL  - total: 

In [17]:
county_sum_check(prec_cong_gdf, bcd_votes, 'COUNTYFP_x', 'COUNTYFP20')

counties to check:  set()
County check complete!


### SLDU Testing
**No splits, join as in st**

In [18]:
set(prec_gen_sldu_gdf['UNIQUE_ID']) - set(prec_prim_sldu_gdf['UNIQUE_ID'])

set()

In [19]:
set(prec_prim_sldu_gdf['UNIQUE_ID']) - set(prec_gen_sldu_gdf['UNIQUE_ID'])

set()

In [20]:
prec_sldu_gdf = prec_gen_sldu_gdf.merge(prec_prim_sldu_gdf, how='outer', on=['UNIQUE_ID', 'geometry'])

#### maup

In [21]:
bsldu = maup_pre_vote_setup(block_gdf, prec_sldu_gdf, 'PREC_SLDU', "SLDU_DIST", "SLDU_DIST_x")

bsldu_mod = assign_pop_to_zero_pop_prec(bsldu, 'PREC_SLDU', 'P0040001-P0050003', 'VAP_MOD_SLDU')

bsldu_votes = assign_votes(
    'VAP_MOD_SLDU', election_cols(prec_sldu_gdf), prec_sldu_gdf, bsldu_mod
)

column_total_check(election_cols(prec_sldu_gdf), bsldu_votes, prec_sldu_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (8941, 97)
0  null assignments in maup series
VAP_MOD_SLDU  added to precinct_gdf based on maup assignment
GSU02RGAV: EQUAL  - total:  block: 122084.00000000001 prec: 122084 diff: 1.4551915228366852e-11
GSU02DODO: EQUAL  - total:  block: 74240.0 prec: 74240 diff: 0.0
GSU04RLAN: EQUAL  - total:  block: 106021.00000000001 prec: 106021 diff: 1.4551915228366852e-11
GSU04DWYE: EQUAL  - total:  block: 68000.0 prec: 68000 diff: 0.0
GSU06RANT: EQUAL  - total:  block: 99095.99999999997 prec: 99096 diff: -2.9103830456733704e-11
GSU06DFOG: EQUAL  - total:  block: 87280.0 prec: 87280 diff: 0.0
GSU08RBLE: EQUAL  - total:  block: 112313.00000000001 prec: 112313 diff: 1.4551915228366852e-11
GSU08DBRO: EQUAL  - total:  block: 74565.0 prec: 74565 diff: 0.0
GSU10DBAL: EQUAL  - total:  block: 58126.0 prec: 58126 diff: 0.0
GSU10RHAC: EQUAL  - total:  block: 109456.0 prec: 109456 diff: 0.0
GSU12RHUF: EQUAL  - total:  block: 129217.99999999997 prec: 1292

In [22]:
county_sum_check(prec_sldu_gdf, bsldu_votes, 'COUNTYFP_x', 'COUNTYFP20')

counties to check:  set()
County check complete!


### SLDL Testing

**misalignment with geoid to dist matching**

In [23]:
df_prec_prim_sldl = gdf_to_df(prec_prim_sldl_gdf)
prec_sldl_gdf = prim_gen_join(prec_gen_sldl_gdf, df_prec_prim_sldl, "SLDL_DIST")

In [24]:
prec_sldl_gdf['UNIQUE_ID_x'].nunique()

9067

#### maup

In [25]:
prec_sldl_gdf['UNIQUE_ID'] = prec_sldl_gdf['UNIQUE_ID_x']

In [26]:
bsldl = maup_pre_vote_setup(block_gdf, prec_sldl_gdf, 'PREC_SLDL', "SLDL_DIST", "SLDL_DIST_x")

bsldl_mod = assign_pop_to_zero_pop_prec(bsldl, 'PREC_SLDL', 'P0040001-P0050003', 'VAP_MOD_SLDL')

bsldl_votes = assign_votes(
    'VAP_MOD_SLDL', election_cols(prec_sldl_gdf), prec_sldl_gdf, bsldl_mod
)

column_total_check(election_cols(prec_sldl_gdf), bsldl_votes, prec_sldl_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (9067, 409)
0  null assignments in maup series
VAP_MOD_SLDL  added to precinct_gdf based on maup assignment
GSL01DTHE: EQUAL  - total:  block: 15835.999999999998 prec: 15836 diff: -1.8189894035458565e-12
GSL01RWIG: EQUAL  - total:  block: 36729.999999999985 prec: 36730 diff: -1.4551915228366852e-11
GSL02DGRA: EQUAL  - total:  block: 16439.999999999996 prec: 16440 diff: -3.637978807091713e-12
GSL02RJOH: EQUAL  - total:  block: 41101.99999999999 prec: 41102 diff: -7.275957614183426e-12
GSL03RGHA: EQUAL  - total:  block: 37677.0 prec: 37677 diff: 0.0
GSL03DJOH: EQUAL  - total:  block: 27094.000000000004 prec: 27094 diff: 3.637978807091713e-12
GSL04RCUP: EQUAL  - total:  block: 38881.99999999999 prec: 38882 diff: -7.275957614183426e-12
GSL05RGIN: EQUAL  - total:  block: 36242.99999999999 prec: 36243 diff: -7.275957614183426e-12
GSL05DWIN: EQUAL  - total:  block: 12554.000000000002 prec: 12554 diff: 1.8189894035458565e-12
GSL06RHAW: EQUA

In [27]:
county_sum_check(prec_sldl_gdf, bsldl_votes, 'COUNTYFP_x', 'COUNTYFP20')

counties to check:  set()
County check complete!


In [28]:
bsldl_votes.columns[~(bsldl_votes.columns.str.contains('PSL'))&~(bsldl_votes.columns.str.contains('GSL'))]

Index(['STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'GEOID20', 'CD116',
       'SLDU18', 'SLDL18', 'P0040001', 'P0050003', 'NAME20', 'ALAND20',
       'AWATER20', 'geometry', 'P0040001-P0050003', 'maup_assignment',
       'PREC_SLDL', 'SLDL_DIST', 'VAP_MOD_SLDL'],
      dtype='object')

## Merge block election files

In [29]:
b_merge = bst_votes.merge(bcd_votes, how='outer', 
                    on=["GEOID20", "STATEFP20", "COUNTYFP20", "TRACTCE20", "BLOCKCE20", "P0040001", "P0050003", 
                        "NAME20", "ALAND20", "AWATER20", "geometry"], suffixes=('_st', '_cd')
                         ).merge(bsldu_votes, how='outer', 
                           on=["GEOID20", "STATEFP20", "COUNTYFP20", "TRACTCE20", "BLOCKCE20", 
                            "P0040001", "P0050003", "NAME20", "ALAND20", "AWATER20", "geometry"], suffixes=('_cd','_slu')
                                ).merge(bsldl_votes, how='outer', 
                           on=["GEOID20", "STATEFP20", "COUNTYFP20", "TRACTCE20", "BLOCKCE20", 
                            "P0040001", "P0050003", "NAME20", "ALAND20", "AWATER20", "geometry"], suffixes=('_slu', '_sll'))
b_merge = b_merge.fillna(0)

In [30]:
b_merge

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,CD116_st,SLDU18_st,SLDL18_st,P0040001,P0050003,NAME20,ALAND20,AWATER20,geometry,P0040001-P0050003_st,maup_assignment_st,PREC_ST,ST_CTY,VAP_MOD_ST,G20PRERTRU,G20PREDBID,G20PRELJOR,G20PREGHAW,GSSCKRKEN,GSSCKDODO,GSSCFDBRU,GSSCFRFRE,P20PREDBEN,P20PREDBID,P20PREDBLO,P20PREDBUT,P20PREDGAB,P20PREDKLO,P20PREDPAT,P20PREDSAN,P20PREDSTE,P20PREDWAR,PSSCKDODO,PSSCFDBRU,P20PRERTRU,PSSCKRKEN,PSSCFRFRE,CD116_cd,SLDU18_cd,SLDL18_cd,P0040001-P0050003_cd,maup_assignment_cd,PREC_CD,CONG_DIST,VAP_MOD_CD,GCON01RCHA,GCON01LKAH,GCON01DSCH,GCON02DCAS,GCON02RWEN,GCON03DBEA,GCON03RRIC,GCON04DFRE,GCON04RJOR,GCON04LPER,GCON05RLAT,GCON05DRUB,GCON06RJOH,GCON06DROB,GCON07RGIB,GCON07LLAP,GCON07DPOT,GCON08RDAV,GCON08DENO,GCON09DKAP,GCON09RWEB,GCON10DTIM,GCON10RTUR,GCON11DFUD,GCON11RGOR,GCON12RBAL,GCON12DSHE,GCON12LSTE,GCON13LFRI,GCON13RHAG,GCON13DRYA,GCON14RJOY,GCON14DMUE,GCON15DNEW,GCON15RSTI,GCON16DGOD,GCON16RGON,PCON01DFOS,PCON01DSCH,PCON02DCAS,PCON03DBEA,PCON03DHAR,PCON04DFRE,PCON04DLAR,PCON04DSIT,PCON05DCAR,PCON05DRED,PCON05DRUB,PCON06DROB,PCON08DENO,PCON08DGUY,PCON09DKAP,PCON09DROS,PCON10DMOY,PCON10DTIM,PCON11DBEL,PCON11DFUD,PCON11DHOO,PCON11DSHA,PCON12DBEL,PCON12DSHE,PCON13DRYA,PCON14DMUE,PCON15DKIL,PCON15DNEW,PCON16DGOD,PCON16DKAR,PCON01RCHA,PCON02RHAR,PCON02RWEN,PCON03RDUL,PCON03RRIC,PCON04RJOR,PCON05RLAT,PCON06RJOH,PCON06RMOR,PCON07RGIB,PCON08RDAV,PCON08RMEE,PCON09RBAR,PCON09RCON,PCON09RCOR,PCON09RWEB,PCON10RAND,PCON10RFLA,PCON10RTUR,PCON11RGOR,PCON11RSCH,PCON11RTAY,PCON12RBAL,PCON12RDAY,PCON13RHAG,PCON13RHEN,PCON13RLYR,PCON13RRMO,PCON13RJMO,PCON13RSAN,PCON13RTRU,PCON14RJOY,PCON14RPIT,PCON15RHUN,PCON15RSTI,PCON16RGON,PCON01LKAH,PCON04LPER,PCON07LLAP,PCON13LFRI,CD116_slu,SLDU18_slu,SLDL18_slu,P0040001-P0050003_slu,maup_assignment_slu,PREC_SLDU,SLDU_DIST,VAP_MOD_SLDU,GSU02RGAV,GSU02DODO,GSU04RLAN,GSU04DWYE,GSU06RANT,GSU06DFOG,GSU08RBLE,GSU08DBRO,GSU10DBAL,GSU10RHAC,GSU12RHUF,GSU12DPOL,GSU14RJOH,GSU14DOTT,GSU16RKUN,GSU16DLET,GSU18RCIR,GSU18DRAD,GSU20DJOH,GSU20RSCH,GSU22DHUN,GSU22RROM,GSU24RDOL,GSU24DJAC,GSU26RREI,GSU26DSWA,GSU28RDOW,GSU28DSYK,GSU30DFLE,GSU30RHOA,GSU32ROBR,GSU32DOBR,PSU02DODO,PSU02DSUB,PSU04DWYE,PSU06DFOG,PSU06DGRI,PSU08DBRO,PSU10DBAL,PSU12DPOL,PSU14DOTT,PSU16DBAI,PSU16DDOU,PSU16DLET,PSU18DRAD,PSU20DJOH,PSU22DJOH,PSU24DJAC,PSU26DSWA,PSU28DSYK,PSU30DFLE,PSU32DOBR,PSU02RGAV,PSU04RKEL,PSU04RLAN,PSU04RWON,PSU06RANT,PSU06RROB,PSU06RSEL,PSU08RBLE,PSU10RHAC,PSU12RHUF,PSU14RJOH,PSU14RUIB,PSU16RKUN,PSU18RCIR,PSU20RSCH,PSU22RBRA,PSU22RFAL,PSU22RHOV,PSU22RREY,PSU22RROM,PSU24RDOL,PSU26RACK,PSU26RREI,PSU28RDOW,PSU30RHOA,PSU32ROBR,PSU32RPOL,CD116_sll,SLDU18_sll,SLDL18_sll,P0040001-P0050003_sll,maup_assignment_sll,PREC_SLDL,SLDL_DIST,VAP_MOD_SLDL,GSL01DTHE,GSL01RWIG,GSL02DGRA,GSL02RJOH,GSL03RGHA,GSL03DJOH,GSL04RCUP,GSL05RGIN,GSL05DWIN,GSL06RHAW,GSL06DROB,GSL07RPAT,GSL07DSWE,GSL08DSMI,GSL09DBOY,GSL09RRUS,GSL10DUPC,GSL11DHOW,GSL12DBRE,GSL12RPOW,GSL13RHAR,GSL13DSKI,GSL14RMCM,GSL14DSWE,GSL15DCRO,GSL15RKUS,GSL16RGRE,GSL16DSMI,GSL17RHAS,GSL17DMIL,GSL18DBOG,GSL18RPAC,GSL19RFRE,GSL19DLIG,GSL20RBAE,GSL20DBRO,GSL21RCOO,GSL21DLIS,GSL22DLEL,GSL23DDAY,GSL23RLAN,GSL24RMAN,GSL24DRUS,GSL25RBUR,GSL25DJAR,GSL26DCRA,GSL27DBIT,GSL27RBRI,GSL28DMIR,GSL28RMON,GSL29RABR,GSL30IROL,GSL30RSEI,GSL31DKEL,GSL32DING,GSL33DDEN,GSL33RHIL,GSL34DSYK,GSL34RTOD,GSL35DGAL,GSL35RSAR,GSL36DSHA,GSL36RYOU,GSL37RBIG,GSL37DWEI,GSL38DCAM,GSL38RROE,GSL39DBLA,GSL39RMUL,GSL40DJAC,GSL40RPLU,GSL41DBER,GSL41RWHI,GSL42RYOU,GSL43DCOX,GSL43RCRE,GSL44DHIC,GSL44RMCM,GSL45RLOG,GSL45DSOB,GSL46RSAL,GSL46DSHE,GSL47DLAR,GSL47RMER,GSL48ROEL,GSL49RHAA,GSL49DWES,GSL50DSIM,GSL50RSTO,GSL51RCAR,GSL52RGRO,GSL52DHOR,GSL53RHAL,GSL53DNOV,GSL54DSHO,GSL54RZEL,GSL55RMAN,GSL55DSTE,GSL56RLAC,GSL56DMIL,GSL57DADK,GSL57RSTE,GSL58DHAG,GSL58RSIM,GSL59RCUT,GSL59DSTA,GSL60RPHI,GSL60DTRO,GSL61RCAL,GSL61DDUD,GSL62RLIP,GSL62DROS,GSL63DBLA,GSL63RLOY,GSL64DOBR,GSL64RYOD,GSL65DDAR,GSL65RSCH,GSL66RBIR,GSL67RJOR,GSL67DMOR,GSL68RCAR,GSL68DMOU,GSL69DBEH,GSL69RRAY,GSL70DBAR,GSL70RKIC,GSL71DCAR,

## Checks

### Population proportions

In [31]:
check_block_to_prec_pop_prop_sum(['PREC_CD', 'VAP_MOD_CD'], 'VAP_MOD_CD', 'PREC_CD')

9089.0
9089


In [32]:
check_block_to_prec_pop_prop_sum(['PREC_SLDL', 'VAP_MOD_SLDL'], 'VAP_MOD_SLDL', 'PREC_SLDL')

9067.0
9067


In [33]:
check_block_to_prec_pop_prop_sum(['PREC_SLDU', 'VAP_MOD_SLDU'], 'VAP_MOD_SLDU', 'PREC_SLDU')

8941.0
8941


In [34]:
check_block_to_prec_pop_prop_sum(['PREC_ST', 'VAP_MOD_ST'], 'VAP_MOD_ST', 'PREC_ST')

8941.0
8941


### Precinct totals

In [35]:
precinct_sum_check(prec_cong_gdf, b_merge, "PREC_CD")
precinct_sum_check(prec_sldu_gdf, b_merge, "PREC_SLDU")
precinct_sum_check(prec_sldl_gdf, b_merge, "PREC_SLDL")
precinct_sum_check(prec_st_gdf, b_merge, "PREC_ST")

precs to check:  set()
Precinct check complete!
precs to check:  set()
Precinct check complete!
precs to check:  set()
Precinct check complete!
precs to check:  set()
Precinct check complete!


## Export gdf

In [36]:
b_merge.columns[b_merge.columns.str.contains('SLDU')]

Index(['SLDU18_st', 'SLDU18_cd', 'SLDU18_slu', 'PREC_SLDU', 'SLDU_DIST',
       'VAP_MOD_SLDU', 'SLDU18_sll'],
      dtype='object')

In [37]:
b_merge['PRECINCTID'] = b_merge['PREC_ST']

In [38]:
def format_gdf(block_votes, election_columns):
    block_votes['VAP_MOD'] = block_votes['P0040001-P0050003_st']
    ordered_cols = ['GEOID20', 'NAME20', 'STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'PRECINCTID',
                    'CONG_DIST', 'SLDL_DIST', 'SLDU_DIST', 'VAP_MOD'] + \
                   election_columns + \
                   ['ALAND20', 'AWATER20', 'geometry']
    block_votes_export = block_votes[ordered_cols]
    return block_votes_export


def create_shp(gdf, shp_name):
    os.mkdir('./' + shp_name)
    gdf.to_file('./' + shp_name + '/' + shp_name + '.shp')
    print(shp_name, 'shapefile created.')

In [39]:
gen_cand_cols = pd.read_csv('./oh_gen_cand_dict.csv')
prim_cand_cols = pd.read_csv('./oh_prim_cand_dict.csv')

In [40]:
gen_cand_dict = pd.Series(gen_cand_cols['Column'].values, gen_cand_cols['Candidate']).to_dict()
prim_cand_dict = pd.Series(prim_cand_cols['Column'].values, prim_cand_cols['Candidate']).to_dict()

In [41]:
block_votes_export = format_gdf(b_merge, list(gen_cand_dict.keys())+list(prim_cand_dict.keys())).round(2)

In [42]:
block_votes_export.head()

,GEOID20,NAME20,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,PRECINCTID,CONG_DIST,SLDL_DIST,SLDU_DIST,VAP_MOD,G20PREDBID,G20PREGHAW,G20PRELJOR,G20PRERTRU,GCON01RCHA,GCON01LKAH,GCON01DSCH,GCON02DCAS,GCON02RWEN,GCON03DBEA,GCON03RRIC,GCON04DFRE,GCON04RJOR,GCON04LPER,GCON05RLAT,GCON05DRUB,GCON06RJOH,GCON06DROB,GCON07RGIB,GCON07LLAP,GCON07DPOT,GCON08RDAV,GCON08DENO,GCON09DKAP,GCON09RWEB,GCON10DTIM,GCON10RTUR,GCON11DFUD,GCON11RGOR,GCON12RBAL,GCON12DSHE,GCON12LSTE,GCON13LFRI,GCON13RHAG,GCON13DRYA,GCON14RJOY,GCON14DMUE,GCON15DNEW,GCON15RSTI,GCON16DGOD,GCON16RGON,GSU02RGAV,GSU02DODO,GSU04RLAN,GSU04DWYE,GSU06RANT,GSU06DFOG,GSU08RBLE,GSU08DBRO,GSU10DBAL,GSU10RHAC,GSU12RHUF,GSU12DPOL,GSU14RJOH,GSU14DOTT,GSU16RKUN,GSU16DLET,GSU18RCIR,GSU18DRAD,GSU20DJOH,GSU20RSCH,GSU22DHUN,GSU22RROM,GSU24RDOL,GSU24DJAC,GSU26RREI,GSU26DSWA,GSU28RDOW,GSU28DSYK,GSU30DFLE,GSU30RHOA,GSU32ROBR,GSU32DOBR,GSL01DTHE,GSL01RWIG,GSL02DGRA,GSL02RJOH,GSL03RGHA,GSL03DJOH,GSL04RCUP,GSL05RGIN,GSL05DWIN,GSL06RHAW,GSL06DROB,GSL07RPAT,GSL07DSWE,GSL08DSMI,GSL09DBOY,GSL09RRUS,GSL10DUPC,GSL11DHOW,GSL12DBRE,GSL12RPOW,GSL13RHAR,GSL13DSKI,GSL14RMCM,GSL14DSWE,GSL15DCRO,GSL15RKUS,GSL16RGRE,GSL16DSMI,GSL17RHAS,GSL17DMIL,GSL18DBOG,GSL18RPAC,GSL19RFRE,GSL19DLIG,GSL20RBAE,GSL20DBRO,GSL21RCOO,GSL21DLIS,GSL22DLEL,GSL23DDAY,GSL23RLAN,GSL24RMAN,GSL24DRUS,GSL25RBUR,GSL25DJAR,GSL26DCRA,GSL27DBIT,GSL27RBRI,GSL28DMIR,GSL28RMON,GSL29RABR,GSL30IROL,GSL30RSEI,GSL31DKEL,GSL32DING,GSL33DDEN,GSL33RHIL,GSL34DSYK,GSL34RTOD,GSL35DGAL,GSL35RSAR,GSL36DSHA,GSL36RYOU,GSL37RBIG,GSL37DWEI,GSL38DCAM,GSL38RROE,GSL39DBLA,GSL39RMUL,GSL40DJAC,GSL40RPLU,GSL41DBER,GSL41RWHI,GSL42RYOU,GSL43DCOX,GSL43RCRE,GSL44DHIC,GSL44RMCM,GSL45RLOG,GSL45DSOB,GSL46RSAL,GSL46DSHE,GSL47DLAR,GSL47RMER,GSL48ROEL,GSL49RHAA,GSL49DWES,GSL50DSIM,GSL50RSTO,GSL51RCAR,GSL52RGRO,GSL52DHOR,GSL53RHAL,GSL53DNOV,GSL54DSHO,GSL54RZEL,GSL55RMAN,GSL55DSTE,GSL56RLAC,GSL56DMIL,GSL57DADK,GSL57RSTE,GSL58DHAG,GSL58RSIM,GSL59RCUT,GSL59DSTA,GSL60RPHI,GSL60DTRO,GSL61RCAL,GSL61DDUD,GSL62RLIP,GSL62DROS,GSL63DBLA,GSL63RLOY,GSL64DOBR,GSL64RYOD,GSL65DDAR,GSL65RSCH,GSL66RBIR,GSL67RJOR,GSL67DMOR,GSL68RCAR,GSL68DMOU,GSL69DBEH,GSL69RRAY,GSL70DBAR,GSL70RKIC,GSL71DCAR,GSL71RFRA,GSL72RHOU,GSL73RLAM,GSL73DMCC,GSL74RDEA,GSL75DCLI,GSL75RPAV,GSL76RGRE,GSL76DWES,GSL77RLAR,GSL77DWIL,GSL78DOWE,GSL78RSTE,GSL79RKOE,GSL79DRIC,GSL80DJON,GSL80RPOW,GSL81RHOO,GSL82RRIE,GSL83RCRO,GSL84RMAN,GSL84DMON,GSL85RVIT,GSL86RRIC,GSL86DROB,GSL87DBAR,GSL87RMCC,GSL88RCLI,GSL88DLIE,GSL89DMIL,GSL89RSWE,GSL90RBAL,GSL91DDAI,GSL91RWIL,GSL92RJOH,GSL92DWOR,GSL93RSTE,GSL94REDW,GSL94DONE,GSL95RJON,GSL96RFER,GSL96LHER,GSL96DOLI,GSL97RHOL,GSL97DSWO,GSL98DBEE,GSL98RHIL,GSL99DDAN,GSL99RART,GSSCKRKEN,GSSCKDODO,GSSCFDBRU,GSSCFRFRE,P20PREDBEN,P20PREDBID,P20PREDBLO,P20PREDBUT,P20PREDGAB,P20PREDKLO,P20PREDPAT,P20PREDSAN,P20PREDSTE,P20PREDWAR,PCON01DFOS,PCON01DSCH,PCON02DCAS,PCON03DBEA,PCON03DHAR,PCON04DFRE,PCON04DLAR,PCON04DSIT,PCON05DCAR,PCON05DRED,PCON05DRUB,PCON06DROB,PCON08DENO,PCON08DGUY,PCON09DKAP,PCON09DROS,PCON10DMOY,PCON10DTIM,PCON11DBEL,PCON11DFUD,PCON11DHOO,PCON11DSHA,PCON12DBEL,PCON12DSHE,PCON13DRYA,PCON14DMUE,PCON15DKIL,PCON15DNEW,PCON16DGOD,PCON16DKAR,PSSCKDODO,PSSCFDBRU,PSU02DODO,PSU02DSUB,PSU04DWYE,PSU06DFOG,PSU06DGRI,PSU08DBRO,PSU10DBAL,PSU12DPOL,PSU14DOTT,PSU16DBAI,PSU16DDOU,PSU16DLET,PSU18DRAD,PSU20DJOH,PSU22DJOH,PSU24DJAC,PSU26DSWA,PSU28DSYK,PSU30DFLE,PSU32DOBR,PSL01DTHE,PSL03DJOH,PSL05DWIN,PSL06DROB,PSL07DSWE,PSL08DSMI,PSL09DBOY,PSL09DSTO,PSL10DBUR,PSL10DUPC,PSL11DHOW,PSL12DBRE,PSL12DROS,PSL13DSKI,PSL14DSWE,PSL15DCRO,PSL16DROM,PSL16DSMI,PSL17DMIL,PSL18DBOG,PSL19DLIG,PSL20DBRO,PSL21DLIS,PSL22DADH,PSL22DLEL,PSL23DDAY,PSL24DRUS,PSL25DJAM,PSL25DJAR,PSL25DJOS,PSL25DMAK,PSL25DNSI,PSL26DCRA,PSL27DBIT,PSL28DMIR,PSL31DKEL,PSL32DING,PSL32DWAS,PSL33DDEN,PSL33DGRA,PSL34DSYK,PSL35DGAL,PSL36DHAR,PSL36DSHA,PSL37DWEI,PSL38DCAM,PSL39DBLA,PSL39DHIC,PSL39DJON,PSL40DJAC,PSL41DBER,PSL43DCOX,PSL45DSOB,PSL46DSHE,PSL47DLAR,PSL49DWES,PSL50DSIM,PSL52DHOR,PSL53DNOV,PSL54DSHO,PSL55DSTE,PSL56DMIL,PSL57DADK,PSL58DHAG,PSL59DSTA,PSL

In [47]:
# Export block file
shp_name = 'oh_20_block'
create_shp(block_votes_export, shp_name)

oh_20_block shapefile created.


# File checks and readme fields

In [442]:
def create_fields_table(race_field_header_0, fields_dict_0, 
                        add_race_field_header_1 = '', fields_dict_1 = {}, 
                        add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {}):
    '''Purpose: Create fields table used in readme based on field dictionary created separately
    Arguments:
        race_field_header_0: include asterisks "***text***" and label first set of fields
        fields_dict_0: the default dictionary for the primary file (statewide)
        add_race_field_header_1: include asterisks to draw attention to section - ex: "***additional_race_file_name_fields***"
        fields_dict_1: additional fields to go under add_race_field_header_1 header
        add_race_field_header_2 and _3: same use as add_race_field_header_1 - include as needed
        fields_dict_2 and _3: same use as fields_dict_1 - include as needed associated with corresponding add_race_field_header section
    '''
    fields_table_data = {'Field Name': ['',race_field_header_0]  + list(fields_dict_0.keys()) +
                         ['',add_race_field_header_1] + list(fields_dict_1.keys()) +
                         ['',add_race_field_header_2] + list(fields_dict_2.keys()) +
                         ['',add_race_field_header_3] + list(fields_dict_3.keys()),
                         'Description': ['',''] + list(fields_dict_0.values()) + 
                         ['',''] + list(fields_dict_1.values()) + 
                         ['',''] + list(fields_dict_2.values()) +
                         ['',''] + list(fields_dict_3.values())}
    fields_table = pd.DataFrame(fields_table_data)
    return fields_table

In [444]:
create_fields_table('Field Name: ', gen_cand_dict, 
                        add_race_field_header_1 = '', fields_dict_1 = {}, 
                        add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {})

,Field Name,Description
0,,
1,Field Name:,
2,G20PREDBID,President and Vice President Joseph R. Biden ...
3,G20PREGHAW,President and Vice President Howie Hawkins an...
4,G20PRELJOR,President and Vice President Jo Jorgensen and...
...,...,...
257,,
258,,
259,,
260,,


In [454]:
block_votes_export.columns[block_votes_export.columns.isin(election_cols(block_votes_export))].map(prim_cand_dict)

Index([                                                           nan,
                                                                  nan,
                                                                  nan,
                                                                  nan,
                                                                  nan,
                                                                  nan,
                                                                  nan,
                                                                  nan,
                                                                  nan,
                                                                  nan,
       ...
               'State Representative - District 92  Mark Johnson (R)',
                'State Representative - District 93  Jeff Halley (R)',
          'State Representative - District 93  Jason C. Stephens (R)',
                'State Representative - District 94  Jay Edwards (

In [453]:
block_votes_export.head()

,GEOID20,NAME20,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,PRECINCTID,CONG_DIST,SLDL_DIST,SLDU_DIST,VAP_MOD,G20PRERTRU,G20PREDBID,G20PRELJOR,G20PREGHAW,GSSCKRKEN,GSSCKDODO,GSSCFDBRU,GSSCFRFRE,P20PREDBEN,P20PREDBID,P20PREDBLO,P20PREDBUT,P20PREDGAB,P20PREDKLO,P20PREDPAT,P20PREDSAN,P20PREDSTE,P20PREDWAR,PSSCKDODO,PSSCFDBRU,P20PRERTRU,PSSCKRKEN,PSSCFRFRE,GCON01RCHA,GCON01LKAH,GCON01DSCH,GCON02DCAS,GCON02RWEN,GCON03DBEA,GCON03RRIC,GCON04DFRE,GCON04RJOR,GCON04LPER,GCON05RLAT,GCON05DRUB,GCON06RJOH,GCON06DROB,GCON07RGIB,GCON07LLAP,GCON07DPOT,GCON08RDAV,GCON08DENO,GCON09DKAP,GCON09RWEB,GCON10DTIM,GCON10RTUR,GCON11DFUD,GCON11RGOR,GCON12RBAL,GCON12DSHE,GCON12LSTE,GCON13LFRI,GCON13RHAG,GCON13DRYA,GCON14RJOY,GCON14DMUE,GCON15DNEW,GCON15RSTI,GCON16DGOD,GCON16RGON,PCON01DFOS,PCON01DSCH,PCON02DCAS,PCON03DBEA,PCON03DHAR,PCON04DFRE,PCON04DLAR,PCON04DSIT,PCON05DCAR,PCON05DRED,PCON05DRUB,PCON06DROB,PCON08DENO,PCON08DGUY,PCON09DKAP,PCON09DROS,PCON10DMOY,PCON10DTIM,PCON11DBEL,PCON11DFUD,PCON11DHOO,PCON11DSHA,PCON12DBEL,PCON12DSHE,PCON13DRYA,PCON14DMUE,PCON15DKIL,PCON15DNEW,PCON16DGOD,PCON16DKAR,PCON01RCHA,PCON02RHAR,PCON02RWEN,PCON03RDUL,PCON03RRIC,PCON04RJOR,PCON05RLAT,PCON06RJOH,PCON06RMOR,PCON07RGIB,PCON08RDAV,PCON08RMEE,PCON09RBAR,PCON09RCON,PCON09RCOR,PCON09RWEB,PCON10RAND,PCON10RFLA,PCON10RTUR,PCON11RGOR,PCON11RSCH,PCON11RTAY,PCON12RBAL,PCON12RDAY,PCON13RHAG,PCON13RHEN,PCON13RLYR,PCON13RRMO,PCON13RJMO,PCON13RSAN,PCON13RTRU,PCON14RJOY,PCON14RPIT,PCON15RHUN,PCON15RSTI,PCON16RGON,PCON01LKAH,PCON04LPER,PCON07LLAP,PCON13LFRI,GSU02RGAV,GSU02DODO,GSU04RLAN,GSU04DWYE,GSU06RANT,GSU06DFOG,GSU08RBLE,GSU08DBRO,GSU10DBAL,GSU10RHAC,GSU12RHUF,GSU12DPOL,GSU14RJOH,GSU14DOTT,GSU16RKUN,GSU16DLET,GSU18RCIR,GSU18DRAD,GSU20DJOH,GSU20RSCH,GSU22DHUN,GSU22RROM,GSU24RDOL,GSU24DJAC,GSU26RREI,GSU26DSWA,GSU28RDOW,GSU28DSYK,GSU30DFLE,GSU30RHOA,GSU32ROBR,GSU32DOBR,PSU02DODO,PSU02DSUB,PSU04DWYE,PSU06DFOG,PSU06DGRI,PSU08DBRO,PSU10DBAL,PSU12DPOL,PSU14DOTT,PSU16DBAI,PSU16DDOU,PSU16DLET,PSU18DRAD,PSU20DJOH,PSU22DJOH,PSU24DJAC,PSU26DSWA,PSU28DSYK,PSU30DFLE,PSU32DOBR,PSU02RGAV,PSU04RKEL,PSU04RLAN,PSU04RWON,PSU06RANT,PSU06RROB,PSU06RSEL,PSU08RBLE,PSU10RHAC,PSU12RHUF,PSU14RJOH,PSU14RUIB,PSU16RKUN,PSU18RCIR,PSU20RSCH,PSU22RBRA,PSU22RFAL,PSU22RHOV,PSU22RREY,PSU22RROM,PSU24RDOL,PSU26RACK,PSU26RREI,PSU28RDOW,PSU30RHOA,PSU32ROBR,PSU32RPOL,GSL01DTHE,GSL01RWIG,GSL02DGRA,GSL02RJOH,GSL03RGHA,GSL03DJOH,GSL04RCUP,GSL05RGIN,GSL05DWIN,GSL06RHAW,GSL06DROB,GSL07RPAT,GSL07DSWE,GSL08DSMI,GSL09DBOY,GSL09RRUS,GSL10DUPC,GSL11DHOW,GSL12DBRE,GSL12RPOW,GSL13RHAR,GSL13DSKI,GSL14RMCM,GSL14DSWE,GSL15DCRO,GSL15RKUS,GSL16RGRE,GSL16DSMI,GSL17RHAS,GSL17DMIL,GSL18DBOG,GSL18RPAC,GSL19RFRE,GSL19DLIG,GSL20RBAE,GSL20DBRO,GSL21RCOO,GSL21DLIS,GSL22DLEL,GSL23DDAY,GSL23RLAN,GSL24RMAN,GSL24DRUS,GSL25RBUR,GSL25DJAR,GSL26DCRA,GSL27DBIT,GSL27RBRI,GSL28DMIR,GSL28RMON,GSL29RABR,GSL30IROL,GSL30RSEI,GSL31DKEL,GSL32DING,GSL33DDEN,GSL33RHIL,GSL34DSYK,GSL34RTOD,GSL35DGAL,GSL35RSAR,GSL36DSHA,GSL36RYOU,GSL37RBIG,GSL37DWEI,GSL38DCAM,GSL38RROE,GSL39DBLA,GSL39RMUL,GSL40DJAC,GSL40RPLU,GSL41DBER,GSL41RWHI,GSL42RYOU,GSL43DCOX,GSL43RCRE,GSL44DHIC,GSL44RMCM,GSL45RLOG,GSL45DSOB,GSL46RSAL,GSL46DSHE,GSL47DLAR,GSL47RMER,GSL48ROEL,GSL49RHAA,GSL49DWES,GSL50DSIM,GSL50RSTO,GSL51RCAR,GSL52RGRO,GSL52DHOR,GSL53RHAL,GSL53DNOV,GSL54DSHO,GSL54RZEL,GSL55RMAN,GSL55DSTE,GSL56RLAC,GSL56DMIL,GSL57DADK,GSL57RSTE,GSL58DHAG,GSL58RSIM,GSL59RCUT,GSL59DSTA,GSL60RPHI,GSL60DTRO,GSL61RCAL,GSL61DDUD,GSL62RLIP,GSL62DROS,GSL63DBLA,GSL63RLOY,GSL64DOBR,GSL64RYOD,GSL65DDAR,GSL65RSCH,GSL66RBIR,GSL67RJOR,GSL67DMOR,GSL68RCAR,GSL68DMOU,GSL69DBEH,GSL69RRAY,GSL70DBAR,GSL70RKIC,GSL71DCAR,GSL71RFRA,GSL72RHOU,GSL73RLAM,GSL73DMCC,GSL74RDEA,GSL75DCLI,GSL75RPAV,GSL76RGRE,GSL76DWES,GSL77RLAR,GSL77DWIL,GSL78DOWE,GSL78RSTE,GSL79RKOE,GSL79DRIC,GSL80DJON,GSL80RPOW,GSL81RHOO,GSL82RRIE,GSL83RCRO,GSL84RMAN,GSL84DMON,GSL85RVIT,GSL86RRIC,GSL86DROB,GSL87DBAR,GSL87RMCC,GSL88RCLI,GSL88DLIE,GSL89DMIL,GSL89RSWE,GSL90RBAL,GSL91DDAI,GSL91RWIL,GSL92RJOH,GSL92DWOR,GSL93RSTE,GSL94REDW,GSL94DONE,GSL95RJON,GS

In [457]:
list(gen_cand_dict.keys())

['G20PREDBID',
 'G20PREGHAW',
 'G20PRELJOR',
 'G20PRERTRU',
 'GCON01RCHA',
 'GCON01LKAH',
 'GCON01DSCH',
 'GCON02DCAS',
 'GCON02RWEN',
 'GCON03DBEA',
 'GCON03RRIC',
 'GCON04DFRE',
 'GCON04RJOR',
 'GCON04LPER',
 'GCON05RLAT',
 'GCON05DRUB',
 'GCON06RJOH',
 'GCON06DROB',
 'GCON07RGIB',
 'GCON07LLAP',
 'GCON07DPOT',
 'GCON08RDAV',
 'GCON08DENO',
 'GCON09DKAP',
 'GCON09RWEB',
 'GCON10DTIM',
 'GCON10RTUR',
 'GCON11DFUD',
 'GCON11RGOR',
 'GCON12RBAL',
 'GCON12DSHE',
 'GCON12LSTE',
 'GCON13LFRI',
 'GCON13RHAG',
 'GCON13DRYA',
 'GCON14RJOY',
 'GCON14DMUE',
 'GCON15DNEW',
 'GCON15RSTI',
 'GCON16DGOD',
 'GCON16RGON',
 'GSU02RGAV',
 'GSU02DODO',
 'GSU04RLAN',
 'GSU04DWYE',
 'GSU06RANT',
 'GSU06DFOG',
 'GSU08RBLE',
 'GSU08DBRO',
 'GSU10DBAL',
 'GSU10RHAC',
 'GSU12RHUF',
 'GSU12DPOL',
 'GSU14RJOH',
 'GSU14DOTT',
 'GSU16RKUN',
 'GSU16DLET',
 'GSU18RCIR',
 'GSU18DRAD',
 'GSU20DJOH',
 'GSU20RSCH',
 'GSU22DHUN',
 'GSU22RROM',
 'GSU24RDOL',
 'GSU24DJAC',
 'GSU26RREI',
 'GSU26DSWA',
 'GSU28RDOW',
 'GSU28

In [450]:
fields_table = create_fields_table('', gen_cand_dict, 'primary: ', prim_cand_dict)
readme_p2 = fields_table.to_string(formatters={'Description':'{{:<{}s}}'.format(fields_table['Description'].str.len().max()).format, 'Field Name':'{{:<{}s}}'.format(fields_table['Field Name'].str.len().max()).format}, index=False)

In [451]:
print(readme_p2)

Field Name                                                                   Description
                                                                                        
                                                                                        
G20PREDBID President and Vice President  Joseph R. Biden and Kamala D. Harris (D)       
G20PREGHAW President and Vice President  Howie Hawkins and Angela Walker                
G20PRELJOR President and Vice President  Jo Jorgensen and Spike Cohen (L)               
G20PRERTRU President and Vice President  Donald J. Trump and Michael R. Pence (R)       
GCON01RCHA Representative to Congress - District 01  Steve Chabot (R)                   
GCON01LKAH Representative to Congress - District 01  Kevin David Kahn (L)               
GCON01DSCH Representative to Congress - District 01  Kate Schroder (D)                  
GCON02DCAS Representative to Congress - District 02  Jaime M. Castle (D)                
GCON02RWEN Representa

# TODO 
precinct name maup assign, congressional districts

join primary gdf general gdf
        geox         geoy
only general geoy
existing prec still have geometry, smaller than before
rows that have na fillna(0)

cd 
prec u --> prec ucd1 prec ucd1 prec ucd2
join on the common prec (ucd1)

subset the prec with cd from the general
grab those prec from the prim and add cd to them

OR

could just make sure all have CD assignment in the prec name and outer join and take geos from general
any NA instances fill with 0

For sake of block file PREC column just have prec name without the dist assignment
**need to be extra careful with crs and removing column**

----

NC (also for NC still need to handle the 0 vote precinct issue)
SN thinks we can do the same thing, but there will be differences in NC with the lines and precinct names

bcong[['GEOID20','PCONG_DIST', 'GCONG_DIST', 'PREC_PCD', 'PREC_GCD']].loc[(bcong['PCONG_DIST']!=bcong['GCONG_DIST'])&
                                                                      (bcong['PCONG_DIST']!='n/a')&
                                                                      (bcong['PREC_GCD'].str.contains('CD-')), 'PCONG_DIST']
                                                                      
Ideally ^ should be 0, not 430 cases     

the method of taking the subset of the precinct and assigning pop zero for the vote allocation 

pcon05 gcon10 occupy same space - only matter pcon10 should have votes - problem to assign pop to zero, otherwise it shouldnt be a problem

can do it at individual level

if prec dont match btwen prim and gen, but

for every value in prec geoids, if does have cd sq bracket in the general, check to see if it exists with or without the cd bracket in the primary
if it exists but without brackets, 
then we know precicnts match 

check if it starts split syntax

for precinct in list_of_prim_precs:
    if precinct + cd_string in list_of_gen_precs:

with subset of issue precincts
modify names of prec to match between prim and gen
get the otherone that exist

outerjoin with only that subset of precincts from the general and remove
new geo column where general geo only used where it effectively joined


say 5 prec in prim file that does not indicate any splits, should have 10 in the general

----
case where prec/county does not match at all regardless of splits between prim and gen, leave it as is


## Congressional

### General

**New Functions**

In [95]:
(bcdg_votes['P0040001-P0050003']<0).any()

True

In [66]:
bcdg = maup_pre_vote_setup(block_gdf, prec_gen_cong_gdf, 'PREC_GCD', "GCONG_DIST", "CONG_DIST")

bcdg_mod = assign_pop_to_zero_pop_prec(bcdg, 'PREC_GCD', 'P0040001-P0050003', 'VAP_MOD_GCD')

bcdg_votes = assign_votes(
    'VAP_MOD_GCD', election_cols(prec_gen_cong_gdf), prec_gen_cong_gdf, bcdg_mod
)

column_total_check(election_cols(prec_gen_cong_gdf), bcdg_votes, prec_gen_cong_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (9089, 48)
0  null assignments in maup series
VAP_MOD_GCD  added to precinct_gdf based on maup assignment
GCON01RCHA: EQUAL  - total:  block: 199559.9999999999 prec: 199560 diff: -8.731149137020111e-11
GCON01LKAH: EQUAL  - total:  block: 13691.999999999998 prec: 13692 diff: -1.8189894035458565e-12
GCON01DSCH: EQUAL  - total:  block: 172022.00000000006 prec: 172022 diff: 5.820766091346741e-11
GCON02DCAS: EQUAL  - total:  block: 146781.0 prec: 146781 diff: 0.0
GCON02RWEN: EQUAL  - total:  block: 230430.00000000003 prec: 230430 diff: 2.9103830456733704e-11
GCON03DBEA: EQUAL  - total:  block: 227420.00000000003 prec: 227420 diff: 2.9103830456733704e-11
GCON03RRIC: EQUAL  - total:  block: 93568.99999999999 prec: 93569 diff: -1.4551915228366852e-11
GCON04DFRE: EQUAL  - total:  block: 101897.0 prec: 101897 diff: 0.0
GCON04RJOR: EQUAL  - total:  block: 235875.00000000003 prec: 235875 diff: 2.9103830456733704e-11
GCON04LPER: EQUAL  - total: 

### Primary

**New Functions**

In [70]:
bcdp = maup_pre_vote_setup(block_gdf, prec_prim_cong_gdf, 'PREC_PCD', "PCONG_DIST", "CONG_DIST")

bcdp_mod = assign_pop_to_zero_pop_prec(bcdp, 'PREC_PCD', 'P0040001-P0050003', 'VAP_MOD_PCD')

bcdp_votes = assign_votes(
    'VAP_MOD_PCD', election_cols(prec_prim_cong_gdf), prec_prim_cong_gdf, bcdp_mod
)

column_total_check(election_cols(prec_prim_cong_gdf), bcdp_votes, prec_prim_cong_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (9067, 80)
0  null assignments in maup series
VAP_MOD_PCD  added to precinct_gdf based on maup assignment
PCON01DFOS: EQUAL  - total:  block: 17520.0 prec: 17520 diff: 0.0
PCON01DSCH: EQUAL  - total:  block: 36579.00000000001 prec: 36579 diff: 7.275957614183426e-12
PCON02DCAS: EQUAL  - total:  block: 40956.00000000001 prec: 40956 diff: 7.275957614183426e-12
PCON03DBEA: EQUAL  - total:  block: 44995.000000000015 prec: 44995 diff: 1.4551915228366852e-11
PCON03DHAR: EQUAL  - total:  block: 21057.0 prec: 21057 diff: 0.0
PCON04DFRE: EQUAL  - total:  block: 18078.0 prec: 18078 diff: 0.0
PCON04DLAR: EQUAL  - total:  block: 8944.0 prec: 8944 diff: 0.0
PCON04DSIT: EQUAL  - total:  block: 11036.999999999998 prec: 11037 diff: -1.8189894035458565e-12
PCON05DCAR: EQUAL  - total:  block: 7842.999999999999 prec: 7843 diff: -9.094947017729282e-13
PCON05DRED: EQUAL  - total:  block: 9079.0 prec: 9079 diff: 0.0
PCON05DRUB: EQUAL  - total:  block: 179

## SLDL

### General

**New Functions**

In [98]:
bsldlg = maup_pre_vote_setup(block_gdf, prec_gen_sldl_gdf, 'PREC_GSLDL', "GSLDL_DIST", "SLDL_DIST")

bsldlg_mod = assign_pop_to_zero_pop_prec(bsldlg, 'PREC_GSLDL', 'P0040001-P0050003', 'VAP_MOD_GSLDL')

bsldlg_votes = assign_votes(
    'VAP_MOD_GSLDL', election_cols(prec_gen_sldl_gdf), prec_gen_sldl_gdf, bsldlg_mod
)

column_total_check(election_cols(prec_gen_sldl_gdf), bsldlg_votes, prec_gen_sldl_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (9067, 188)
0  null assignments in maup series
VAP_MOD_GSLDL  added to precinct_gdf based on maup assignment
GSL01DTHE: EQUAL  - total:  block: 15835.999999999998 prec: 15836 diff: -1.8189894035458565e-12
GSL01RWIG: EQUAL  - total:  block: 36729.999999999985 prec: 36730 diff: -1.4551915228366852e-11
GSL02DGRA: EQUAL  - total:  block: 16439.999999999996 prec: 16440 diff: -3.637978807091713e-12
GSL02RJOH: EQUAL  - total:  block: 41101.99999999999 prec: 41102 diff: -7.275957614183426e-12
GSL03RGHA: EQUAL  - total:  block: 37677.0 prec: 37677 diff: 0.0
GSL03DJOH: EQUAL  - total:  block: 27094.000000000004 prec: 27094 diff: 3.637978807091713e-12
GSL04RCUP: EQUAL  - total:  block: 38881.99999999999 prec: 38882 diff: -7.275957614183426e-12
GSL05RGIN: EQUAL  - total:  block: 36242.99999999999 prec: 36243 diff: -7.275957614183426e-12
GSL05DWIN: EQUAL  - total:  block: 12554.000000000002 prec: 12554 diff: 1.8189894035458565e-12
GSL06RHAW: EQU

In [99]:
county_sum_check(prec_gen_sldl_gdf, bsldlg_votes, 'COUNTYFP', 'COUNTYFP20')

counties to check:  set()
County check complete!


### Primary

**New Functions**

In [80]:
bsldlp = maup_pre_vote_setup(block_gdf, prec_prim_sldl_gdf, 'PREC_PSLDL', "PSLDL_DIST", "SLDL_DIST")

bsldlp_mod = assign_pop_to_zero_pop_prec(bsldlp, 'PREC_PSLDL', 'P0040001-P0050003', 'VAP_MOD_PSLDL')

bsldlp_votes = assign_votes(
    'VAP_MOD_PSLDL', election_cols(prec_prim_sldl_gdf), prec_prim_sldl_gdf, bsldlp_mod
)

column_total_check(election_cols(prec_prim_sldl_gdf), bsldlp_votes, prec_prim_sldl_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (9033, 221)
0  null assignments in maup series
VAP_MOD_PSLDL  added to precinct_gdf based on maup assignment
PSL01DTHE: EQUAL  - total:  block: 4253.000000000001 prec: 4253 diff: 9.094947017729282e-13
PSL03DJOH: EQUAL  - total:  block: 7243.0 prec: 7243 diff: 0.0
PSL05DWIN: EQUAL  - total:  block: 3911.0000000000005 prec: 3911 diff: 4.547473508864641e-13
PSL06DROB: EQUAL  - total:  block: 10753.000000000002 prec: 10753 diff: 1.8189894035458565e-12
PSL07DSWE: EQUAL  - total:  block: 8082.000000000001 prec: 8082 diff: 9.094947017729282e-13
PSL08DSMI: EQUAL  - total:  block: 11184.999999999998 prec: 11185 diff: -1.8189894035458565e-12
PSL09DBOY: EQUAL  - total:  block: 16007.999999999995 prec: 16008 diff: -5.4569682106375694e-12
PSL09DSTO: EQUAL  - total:  block: 2580.0 prec: 2580 diff: 0.0
PSL10DBUR: EQUAL  - total:  block: 1355.0 prec: 1355 diff: 0.0
PSL10DUPC: EQUAL  - total:  block: 6151.000000000001 prec: 6151 diff: 9.094947017729

In [81]:
county_sum_check(prec_prim_sldl_gdf, bsldlp_votes, 'COUNTYFP', 'COUNTYFP20')

counties to check:  set()
County check complete!


## SLDU

### General

**New Functions**

In [83]:
bsldug = maup_pre_vote_setup(block_gdf, prec_gen_sldu_gdf, 'PREC_GSLDU', "GSLDU_DIST", "SLDU_DIST")

bsldug_mod = assign_pop_to_zero_pop_prec(bsldug, 'PREC_GSLDU', 'P0040001-P0050003', 'VAP_MOD_GSLDU')

bsldug_votes = assign_votes(
    'VAP_MOD_GSLDU', election_cols(prec_gen_sldu_gdf), prec_gen_sldu_gdf, bsldug_mod
)

column_total_check(election_cols(prec_gen_sldu_gdf), bsldug_votes, prec_gen_sldu_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (8941, 42)
0  null assignments in maup series
VAP_MOD_GSLDU  added to precinct_gdf based on maup assignment
GSU02RGAV: EQUAL  - total:  block: 122084.00000000001 prec: 122084 diff: 1.4551915228366852e-11
GSU02DODO: EQUAL  - total:  block: 74240.0 prec: 74240 diff: 0.0
GSU04RLAN: EQUAL  - total:  block: 106021.00000000001 prec: 106021 diff: 1.4551915228366852e-11
GSU04DWYE: EQUAL  - total:  block: 68000.0 prec: 68000 diff: 0.0
GSU06RANT: EQUAL  - total:  block: 99095.99999999997 prec: 99096 diff: -2.9103830456733704e-11
GSU06DFOG: EQUAL  - total:  block: 87280.0 prec: 87280 diff: 0.0
GSU08RBLE: EQUAL  - total:  block: 112313.00000000001 prec: 112313 diff: 1.4551915228366852e-11
GSU08DBRO: EQUAL  - total:  block: 74565.0 prec: 74565 diff: 0.0
GSU10DBAL: EQUAL  - total:  block: 58126.0 prec: 58126 diff: 0.0
GSU10RHAC: EQUAL  - total:  block: 109456.0 prec: 109456 diff: 0.0
GSU12RHUF: EQUAL  - total:  block: 129217.99999999997 prec: 129

In [84]:
county_sum_check(prec_gen_sldu_gdf, bsldug_votes, 'COUNTYFP', 'COUNTYFP20')

counties to check:  set()
County check complete!


### Primary

**New Functions**

In [85]:
bsldup = maup_pre_vote_setup(block_gdf, prec_prim_sldu_gdf, 'PREC_PSLDU', "PSLDU_DIST", "SLDU_DIST")

bsldup_mod = assign_pop_to_zero_pop_prec(bsldup, 'PREC_PSLDU', 'P0040001-P0050003', 'VAP_MOD_PSLDU')

bsldup_votes = assign_votes(
    'VAP_MOD_PSLDU', election_cols(prec_prim_sldu_gdf), prec_prim_sldu_gdf, bsldup_mod
)

column_total_check(election_cols(prec_prim_sldu_gdf), bsldup_votes, prec_prim_sldu_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (8941, 57)
0  null assignments in maup series
VAP_MOD_PSLDU  added to precinct_gdf based on maup assignment
PSU02DODO: EQUAL  - total:  block: 12169.999999999998 prec: 12170 diff: -1.8189894035458565e-12
PSU02DSUB: EQUAL  - total:  block: 11162.0 prec: 11162 diff: 0.0
PSU04DWYE: EQUAL  - total:  block: 12568.0 prec: 12568 diff: 0.0
PSU06DFOG: EQUAL  - total:  block: 16867.0 prec: 16867 diff: 0.0
PSU06DGRI: EQUAL  - total:  block: 3643.0000000000005 prec: 3643 diff: 4.547473508864641e-13
PSU08DBRO: EQUAL  - total:  block: 18091.0 prec: 18091 diff: 0.0
PSU10DBAL: EQUAL  - total:  block: 16232.0 prec: 16232 diff: 0.0
PSU12DPOL: EQUAL  - total:  block: 10900.0 prec: 10900 diff: 0.0
PSU14DOTT: EQUAL  - total:  block: 13059.999999999998 prec: 13060 diff: -1.8189894035458565e-12
PSU16DBAI: EQUAL  - total:  block: 1880.0000000000002 prec: 1880 diff: 2.2737367544323206e-13
PSU16DDOU: EQUAL  - total:  block: 4389.0 prec: 4389 diff: 0.0
PSU16D

In [86]:
county_sum_check(prec_prim_sldu_gdf, bsldup_votes, 'COUNTYFP', 'COUNTYFP20')

counties to check:  set()
County check complete!


## State

### General

**New Functions**

In [88]:
bstg = maup_pre_vote_setup(block_gdf, prec_gen_st_gdf, 'PREC_GST', "GST_CTY", "COUNTYFP")

bstg_mod = assign_pop_to_zero_pop_prec(bstg, 'PREC_GST', 'P0040001-P0050003', 'VAP_MOD_GST')

bstg_votes = assign_votes(
    'VAP_MOD_GST', election_cols(prec_gen_st_gdf), prec_gen_st_gdf, bstg_mod
)

column_total_check(election_cols(prec_gen_st_gdf), bstg_votes, prec_gen_st_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (8941, 18)
0  null assignments in maup series
VAP_MOD_GST  added to precinct_gdf based on maup assignment
G20PRERTRU: EQUAL  - total:  block: 3154834.0 prec: 3154834 diff: 0.0
G20PREDBID: EQUAL  - total:  block: 2679165.0000000005 prec: 2679165 diff: 4.656612873077393e-10
G20PRELJOR: EQUAL  - total:  block: 67568.99999999999 prec: 67569 diff: -1.4551915228366852e-11
G20PREGHAW: EQUAL  - total:  block: 18812.000000000004 prec: 18812 diff: 3.637978807091713e-12
GSSCKRKEN: EQUAL  - total:  block: 2735041.0000000005 prec: 2735041 diff: 4.656612873077393e-10
GSSCKDODO: EQUAL  - total:  block: 2231724.0000000005 prec: 2231724 diff: 4.656612873077393e-10
GSSCFDBRU: EQUAL  - total:  block: 2695072.0 prec: 2695072 diff: 0.0
GSSCFRFRE: EQUAL  - total:  block: 2174820.0 prec: 2174820 diff: 0.0


In [91]:
county_sum_check(prec_gen_st_gdf, bstg_votes, 'COUNTYFP', 'COUNTYFP20')

counties to check:  set()
County check complete!


### Primary

**New Functions**

In [90]:
bstp = maup_pre_vote_setup(block_gdf, prec_prim_st_gdf, 'PREC_PST', "PST_CTY", "COUNTYFP")

bstp_mod = assign_pop_to_zero_pop_prec(bstp, 'PREC_PST', 'P0040001-P0050003', 'VAP_MOD_PST')

bstp_votes = assign_votes(
    'VAP_MOD_PST', election_cols(prec_prim_st_gdf), prec_prim_st_gdf, bstp_mod
)

column_total_check(election_cols(prec_prim_st_gdf), bstp_votes, prec_prim_st_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (8941, 25)
0  null assignments in maup series
VAP_MOD_PST  added to precinct_gdf based on maup assignment
P20PREDBEN: EQUAL  - total:  block: 2030.0 prec: 2030 diff: 0.0
P20PREDBID: EQUAL  - total:  block: 647284.0000000001 prec: 647284 diff: 1.1641532182693481e-10
P20PREDBLO: EQUAL  - total:  block: 28703.999999999996 prec: 28704 diff: -3.637978807091713e-12
P20PREDBUT: EQUAL  - total:  block: 15113.000000000004 prec: 15113 diff: 3.637978807091713e-12
P20PREDGAB: EQUAL  - total:  block: 4559.999999999999 prec: 4560 diff: -9.094947017729282e-13
P20PREDKLO: EQUAL  - total:  block: 11899.0 prec: 11899 diff: 0.0
P20PREDPAT: EQUAL  - total:  block: 822.0000000000002 prec: 822 diff: 2.2737367544323206e-13
P20PREDSAN: EQUAL  - total:  block: 149682.99999999997 prec: 149683 diff: -2.9103830456733704e-11
P20PREDSTE: EQUAL  - total:  block: 2801.0000000000005 prec: 2801 diff: 4.547473508864641e-13
P20PREDWAR: EQUAL  - total:  block: 30984.99

In [92]:
county_sum_check(prec_prim_st_gdf, bstp_votes, 'COUNTYFP', 'COUNTYFP20')

counties to check:  set()
County check complete!


# Merge

In [144]:
b_merge = bsldup_votes.merge(bsldug_votes, 
             how='outer', 
             on=["GEOID20", "STATEFP20", "COUNTYFP20", "TRACTCE20", "BLOCKCE20", "P0040001", "P0050003", 
                 "NAME20", "ALAND20", "AWATER20", "geometry", "P0040001-P0050003"], 
             suffixes=('_sldup','_sldug')
            ).merge(bsldlp_votes, how='outer', 
                    on=["GEOID20", "STATEFP20", "COUNTYFP20", "TRACTCE20", "BLOCKCE20", "P0040001", "P0050003", 
                        "NAME20", "ALAND20", "AWATER20", "geometry"]
                   ).merge(bsldlg_votes, 
                           how='outer', 
                           on=["GEOID20", "STATEFP20", "COUNTYFP20", "TRACTCE20", "BLOCKCE20", "P0040001", "P0050003", 
                                "NAME20", "ALAND20", "AWATER20", "geometry"], suffixes=('_sldlp','_sldlg')
                          ).merge(bcdp_votes, 
                                  how='outer', 
                                   on=["GEOID20", "STATEFP20", "COUNTYFP20", "TRACTCE20", "BLOCKCE20", "P0040001", "P0050003", 
                                        "NAME20", "ALAND20", "AWATER20", "geometry"]
                                 ).merge(bcdg_votes, 
                                         how='outer', 
                                         on=["GEOID20", "STATEFP20", "COUNTYFP20", "TRACTCE20", "BLOCKCE20", 
                                             "P0040001", "P0050003", "NAME20", "ALAND20", "AWATER20", "geometry"], suffixes=('_cdp','_cdg')
                                        ).merge(bstp_votes, 
                                             how='outer', 
                                             on=["GEOID20", "STATEFP20", "COUNTYFP20", "TRACTCE20", "BLOCKCE20", 
                                             "P0040001", "P0050003", "NAME20", "ALAND20", "AWATER20", "geometry"]
                                            ).merge(bstg_votes, 
                                                 how='outer', 
                                                on=["GEOID20", "STATEFP20", "COUNTYFP20", "TRACTCE20", "BLOCKCE20", 
                                                 "P0040001", "P0050003", "NAME20", "ALAND20", "AWATER20", "geometry"], suffixes=('_stp','_stg')
                                        )
b_merge = b_merge.fillna(0)

In [145]:
b_merge

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,CD116_sldup,SLDU18_sldup,SLDL18_sldup,P0040001,P0050003,NAME20,ALAND20,AWATER20,geometry,P0040001-P0050003_x,maup_assignment_sldup,PREC_PSLDU,PSLDU_DIST,VAP_MOD_PSLDU,PSU02DODO,PSU02DSUB,PSU04DWYE,PSU06DFOG,PSU06DGRI,PSU08DBRO,PSU10DBAL,PSU12DPOL,PSU14DOTT,PSU16DBAI,PSU16DDOU,PSU16DLET,PSU18DRAD,PSU20DJOH,PSU22DJOH,PSU24DJAC,PSU26DSWA,PSU28DSYK,PSU30DFLE,PSU32DOBR,PSU02RGAV,PSU04RKEL,PSU04RLAN,PSU04RWON,PSU06RANT,PSU06RROB,PSU06RSEL,PSU08RBLE,PSU10RHAC,PSU12RHUF,PSU14RJOH,PSU14RUIB,PSU16RKUN,PSU18RCIR,PSU20RSCH,PSU22RBRA,PSU22RFAL,PSU22RHOV,PSU22RREY,PSU22RROM,PSU24RDOL,PSU26RACK,PSU26RREI,PSU28RDOW,PSU30RHOA,PSU32ROBR,PSU32RPOL,CD116_sldug,SLDU18_sldug,SLDL18_sldug,maup_assignment_sldug,PREC_GSLDU,GSLDU_DIST,VAP_MOD_GSLDU,GSU02RGAV,GSU02DODO,GSU04RLAN,GSU04DWYE,GSU06RANT,GSU06DFOG,GSU08RBLE,GSU08DBRO,GSU10DBAL,GSU10RHAC,GSU12RHUF,GSU12DPOL,GSU14RJOH,GSU14DOTT,GSU16RKUN,GSU16DLET,GSU18RCIR,GSU18DRAD,GSU20DJOH,GSU20RSCH,GSU22DHUN,GSU22RROM,GSU24RDOL,GSU24DJAC,GSU26RREI,GSU26DSWA,GSU28RDOW,GSU28DSYK,GSU30DFLE,GSU30RHOA,GSU32ROBR,GSU32DOBR,CD116_sldlp,SLDU18_sldlp,SLDL18_sldlp,P0040001-P0050003_y,maup_assignment_sldlp,PREC_PSLDL,PSLDL_DIST,VAP_MOD_PSLDL,PSL01DTHE,PSL03DJOH,PSL05DWIN,PSL06DROB,PSL07DSWE,PSL08DSMI,PSL09DBOY,PSL09DSTO,PSL10DBUR,PSL10DUPC,PSL11DHOW,PSL12DBRE,PSL12DROS,PSL13DSKI,PSL14DSWE,PSL15DCRO,PSL16DROM,PSL16DSMI,PSL17DMIL,PSL18DBOG,PSL19DLIG,PSL20DBRO,PSL21DLIS,PSL22DADH,PSL22DLEL,PSL23DDAY,PSL24DRUS,PSL25DJAM,PSL25DJAR,PSL25DJOS,PSL25DMAK,PSL25DNSI,PSL26DCRA,PSL27DBIT,PSL28DMIR,PSL31DKEL,PSL32DING,PSL32DWAS,PSL33DDEN,PSL33DGRA,PSL34DSYK,PSL35DGAL,PSL36DHAR,PSL36DSHA,PSL37DWEI,PSL38DCAM,PSL39DBLA,PSL39DHIC,PSL39DJON,PSL40DJAC,PSL41DBER,PSL43DCOX,PSL45DSOB,PSL46DSHE,PSL47DLAR,PSL49DWES,PSL50DSIM,PSL52DHOR,PSL53DNOV,PSL54DSHO,PSL55DSTE,PSL56DMIL,PSL57DADK,PSL58DHAG,PSL59DSTA,PSL60DTRO,PSL61DDUD,PSL62DROS,PSL63DBLA,PSL63DLAN,PSL63DPRO,PSL64DOBR,PSL65DDAR,PSL67DMOR,PSL68DMOU,PSL69DBEH,PSL70DBAR,PSL71DCAR,PSL73DMCC,PSL75DCLI,PSL76DWES,PSL77DWIL,PSL78DOWE,PSL79DRIC,PSL84DMON,PSL86DROB,PSL87DBAR,PSL88DLIE,PSL89DMIL,PSL91DDAI,PSL92DWOR,PSL94DONE,PSL96DDIP,PSL96DOLI,PSL97DSWO,PSL98DBEE,PSL99DDAN,PSL01RWET,PSL01RWIG,PSL02RJOH,PSL02RMAR,PSL03RGHA,PSL04RCUP,PSL05RGIN,PSL06RHAR,PSL06RHAW,PSL07RPAT,PSL09RRUS,PSL14RMCM,PSL15RKUS,PSL16RGRE,PSL17RHAS,PSL17RRUT,PSL18RPAC,PSL19RFRE,PSL20RBAE,PSL21RCOO,PSL22RSPE,PSL23RLAN,PSL24RMAN,PSL25RBUR,PSL27RBRI,PSL28RMON,PSL29RABR,PSL30RSEI,PSL33RHIL,PSL34RTOD,PSL35RSAR,PSL36RYOU,PSL37RBIG,PSL38RROE,PSL39RMUL,PSL40RPLU,PSL41RWHI,PSL42RSTU,PSL42RYOU,PSL43RCRE,PSL43RSMI,PSL44RMCM,PSL45RLOG,PSL46RSAL,PSL47RMER,PSL48ROEL,PSL49RHAA,PSL50RSTO,PSL51RCAR,PSL52RGRO,PSL52RWEL,PSL53RGUI,PSL53RHAL,PSL53RMUL,PSL54RZEL,PSL55RMAN,PSL56RLAC,PSL57RSTE,PSL58RSIM,PSL59RMAN,PSL60RPHI,PSL61RCAL,PSL62RLIP,PSL63RLOY,PSL64RYOD,PSL65RBLE,PSL65RDIL,PSL65RSCH,PSL66RBIR,PSL66RFRE,PSL66ROWE,PSL67RJOR,PSL68RCAR,PSL69RRAY,PSL70RKIC,PSL70RROB,PSL71RCLA,PSL71RFRA,PSL72RHOU,PSL73RBRO,PSL73RLAM,PSL74RDEA,PSL75RPAV,PSL76RGRE,PSL76RHAL,PSL77RLAR,PSL78RADA,PSL78RMIT,PSL78RSTE,PSL79RKOE,PSL80RPOW,PSL81RHOO,PSL82RRIE,PSL83RCRO,PSL84RMAN,PSL85RVIT,PSL86RRIC,PSL87RMCC,PSL88RCLI,PSL88ROLL,PSL88RTHO,PSL89RSWE,PSL90RBAL,PSL91RWIL,PSL92RJOH,PSL93RHAL,PSL93RSTE,PSL94REDW,PSL95RJON,PSL96RFER,PSL97RHOL,PSL98RHIL,PSL99RFOW,PSL96LHER,CD116_sldlg,SLDU18_sldlg,SLDL18_sldlg,P0040001-P0050003_x,maup_assignment_sldlg,PREC_GSLDL,GSLDL_DIST,VAP_MOD_GSLDL,GSL01DTHE,GSL01RWIG,GSL02DGRA,GSL02RJOH,GSL03RGHA,GSL03DJOH,GSL04RCUP,GSL05RGIN,GSL05DWIN,GSL06RHAW,GSL06DROB,GSL07RPAT,GSL07DSWE,GSL08DSMI,GSL09DBOY,GSL09RRUS,GSL10DUPC,GSL11DHOW,GSL12DBRE,GSL12RPOW,GSL13RHAR,GSL13DSKI,GSL14RMCM,GSL14DSWE,GSL15DCRO,GSL15RKUS,GSL16RGRE,GSL16DSMI,GSL17RHAS,GSL17DMIL,GSL18DBOG,GSL18RPAC,GSL19RFRE,GSL19DLIG,GSL20RBAE,GSL20DBRO,GSL21RCOO,GSL21DLIS,GSL22DLEL,GSL23DDAY,GSL23RLAN,GSL24RMAN,GSL24DRUS,GSL25RBUR,GSL25DJAR,GSL26DCRA,GSL27DBIT,GSL27RBRI,GSL28DMIR,GSL28RMON,GSL29RABR,GSL30IROL,GSL30RSEI,GSL31DKEL,GSL32DING,GSL33

# Checks

## Population proportions

In [156]:
def check_block_to_prec_pop_prop_sum(pop_col_list, VAP_MOD_COL, PREC_COL):
    pop_col_list = pop_col_list + ['GEOID20', 'P0040001', 'P0050003', 'P0040001-P0050003']
    popcd = b_merge[pop_col_list]
    prec_popcd = popcd.groupby(PREC_COL).sum().reset_index()

    prec_to_pop_dict_gcd = pd.Series(prec_popcd[VAP_MOD_COL].values, index = prec_popcd[PREC_COL]).to_dict()

    popcd['prec_popgcd'] = popcd[PREC_COL].map(prec_to_pop_dict_gcd)
    popcd['propgcd'] = popcd[VAP_MOD_COL]/popcd['prec_popgcd']

    print(popcd.groupby(PREC_COL).sum()['propgcd'].sum())
    print(popcd[PREC_COL].nunique())

In [154]:
check_block_to_prec_pop_prop_sum(['PREC_GCD', 'VAP_MOD_GCD'], 'VAP_MOD_GCD', 'PREC_GCD')

9089.0
9089


In [157]:
check_block_to_prec_pop_prop_sum(['PREC_PCD', 'VAP_MOD_PCD'], 'VAP_MOD_PCD', 'PREC_PCD')

9067.0
9067


In [158]:
check_block_to_prec_pop_prop_sum(['PREC_GSLDL', 'VAP_MOD_GSLDL'], 'VAP_MOD_GSLDL', 'PREC_GSLDL')

9067.0
9067


In [159]:
check_block_to_prec_pop_prop_sum(['PREC_PSLDL', 'VAP_MOD_PSLDL'], 'VAP_MOD_PSLDL', 'PREC_PSLDL')

9033.0
9033


In [160]:
check_block_to_prec_pop_prop_sum(['PREC_GSLDU', 'VAP_MOD_GSLDU'], 'VAP_MOD_GSLDU', 'PREC_GSLDU')

8941.0
8941


In [161]:
check_block_to_prec_pop_prop_sum(['PREC_PSLDU', 'VAP_MOD_PSLDU'], 'VAP_MOD_PSLDU', 'PREC_PSLDU')

8941.0
8941


In [162]:
check_block_to_prec_pop_prop_sum(['PREC_GST', 'VAP_MOD_GST'], 'VAP_MOD_GST', 'PREC_GST')

8941.0
8941


In [163]:
check_block_to_prec_pop_prop_sum(['PREC_PST', 'VAP_MOD_PST'], 'VAP_MOD_PST', 'PREC_PST')

8941.0
8941


## Column totals

In [146]:
column_total_check(election_cols(prec_gen_cong_gdf), b_merge, prec_gen_cong_gdf)

GCON01RCHA: EQUAL  - total:  block: 199559.9999999999 prec: 199560 diff: -8.731149137020111e-11
GCON01LKAH: EQUAL  - total:  block: 13691.999999999998 prec: 13692 diff: -1.8189894035458565e-12
GCON01DSCH: EQUAL  - total:  block: 172022.00000000006 prec: 172022 diff: 5.820766091346741e-11
GCON02DCAS: EQUAL  - total:  block: 146781.0 prec: 146781 diff: 0.0
GCON02RWEN: EQUAL  - total:  block: 230430.00000000003 prec: 230430 diff: 2.9103830456733704e-11
GCON03DBEA: EQUAL  - total:  block: 227420.00000000003 prec: 227420 diff: 2.9103830456733704e-11
GCON03RRIC: EQUAL  - total:  block: 93568.99999999999 prec: 93569 diff: -1.4551915228366852e-11
GCON04DFRE: EQUAL  - total:  block: 101897.0 prec: 101897 diff: 0.0
GCON04RJOR: EQUAL  - total:  block: 235875.00000000003 prec: 235875 diff: 2.9103830456733704e-11
GCON04LPER: EQUAL  - total:  block: 9854.000000000002 prec: 9854 diff: 1.8189894035458565e-12
GCON05RLAT: EQUAL  - total:  block: 257019.0 prec: 257019 diff: 0.0
GCON05DRUB: EQUAL  - total

In [148]:
column_total_check(election_cols(prec_prim_cong_gdf), b_merge, prec_prim_cong_gdf)

PCON01DFOS: EQUAL  - total:  block: 17520.0 prec: 17520 diff: 0.0
PCON01DSCH: EQUAL  - total:  block: 36579.00000000001 prec: 36579 diff: 7.275957614183426e-12
PCON02DCAS: EQUAL  - total:  block: 40956.00000000001 prec: 40956 diff: 7.275957614183426e-12
PCON03DBEA: EQUAL  - total:  block: 44995.000000000015 prec: 44995 diff: 1.4551915228366852e-11
PCON03DHAR: EQUAL  - total:  block: 21057.0 prec: 21057 diff: 0.0
PCON04DFRE: EQUAL  - total:  block: 18078.0 prec: 18078 diff: 0.0
PCON04DLAR: EQUAL  - total:  block: 8944.0 prec: 8944 diff: 0.0
PCON04DSIT: EQUAL  - total:  block: 11036.999999999998 prec: 11037 diff: -1.8189894035458565e-12
PCON05DCAR: EQUAL  - total:  block: 7842.999999999999 prec: 7843 diff: -9.094947017729282e-13
PCON05DRED: EQUAL  - total:  block: 9079.0 prec: 9079 diff: 0.0
PCON05DRUB: EQUAL  - total:  block: 17902.0 prec: 17902 diff: 0.0
PCON06DROB: EQUAL  - total:  block: 30627.999999999996 prec: 30628 diff: -3.637978807091713e-12
PCON08DENO: EQUAL  - total:  block: 2

In [149]:
column_total_check(election_cols(prec_gen_sldl_gdf), b_merge, prec_gen_sldl_gdf)

GSL01DTHE: EQUAL  - total:  block: 15835.999999999998 prec: 15836 diff: -1.8189894035458565e-12
GSL01RWIG: EQUAL  - total:  block: 36729.999999999985 prec: 36730 diff: -1.4551915228366852e-11
GSL02DGRA: EQUAL  - total:  block: 16439.999999999996 prec: 16440 diff: -3.637978807091713e-12
GSL02RJOH: EQUAL  - total:  block: 41101.99999999999 prec: 41102 diff: -7.275957614183426e-12
GSL03RGHA: EQUAL  - total:  block: 37677.0 prec: 37677 diff: 0.0
GSL03DJOH: EQUAL  - total:  block: 27094.000000000004 prec: 27094 diff: 3.637978807091713e-12
GSL04RCUP: EQUAL  - total:  block: 38881.99999999999 prec: 38882 diff: -7.275957614183426e-12
GSL05RGIN: EQUAL  - total:  block: 36242.99999999999 prec: 36243 diff: -7.275957614183426e-12
GSL05DWIN: EQUAL  - total:  block: 12554.000000000002 prec: 12554 diff: 1.8189894035458565e-12
GSL06RHAW: EQUAL  - total:  block: 35982.00000000001 prec: 35982 diff: 7.275957614183426e-12
GSL06DROB: EQUAL  - total:  block: 37995.00000000001 prec: 37995 diff: 7.27595761418

In [150]:
column_total_check(election_cols(prec_prim_sldl_gdf), b_merge, prec_prim_sldl_gdf)

PSL01DTHE: EQUAL  - total:  block: 4253.000000000001 prec: 4253 diff: 9.094947017729282e-13
PSL03DJOH: EQUAL  - total:  block: 7243.0 prec: 7243 diff: 0.0
PSL05DWIN: EQUAL  - total:  block: 3911.0000000000005 prec: 3911 diff: 4.547473508864641e-13
PSL06DROB: EQUAL  - total:  block: 10753.000000000002 prec: 10753 diff: 1.8189894035458565e-12
PSL07DSWE: EQUAL  - total:  block: 8082.000000000001 prec: 8082 diff: 9.094947017729282e-13
PSL08DSMI: EQUAL  - total:  block: 11184.999999999998 prec: 11185 diff: -1.8189894035458565e-12
PSL09DBOY: EQUAL  - total:  block: 16007.999999999995 prec: 16008 diff: -5.4569682106375694e-12
PSL09DSTO: EQUAL  - total:  block: 2580.0 prec: 2580 diff: 0.0
PSL10DBUR: EQUAL  - total:  block: 1355.0 prec: 1355 diff: 0.0
PSL10DUPC: EQUAL  - total:  block: 6151.000000000001 prec: 6151 diff: 9.094947017729282e-13
PSL11DHOW: EQUAL  - total:  block: 5760.0 prec: 5760 diff: 0.0
PSL12DBRE: EQUAL  - total:  block: 10014.999999999998 prec: 10015 diff: -1.8189894035458565e

In [151]:
column_total_check(election_cols(prec_gen_sldu_gdf), b_merge, prec_gen_sldu_gdf)

GSU02RGAV: EQUAL  - total:  block: 122084.00000000001 prec: 122084 diff: 1.4551915228366852e-11
GSU02DODO: EQUAL  - total:  block: 74240.0 prec: 74240 diff: 0.0
GSU04RLAN: EQUAL  - total:  block: 106021.00000000001 prec: 106021 diff: 1.4551915228366852e-11
GSU04DWYE: EQUAL  - total:  block: 68000.0 prec: 68000 diff: 0.0
GSU06RANT: EQUAL  - total:  block: 99095.99999999997 prec: 99096 diff: -2.9103830456733704e-11
GSU06DFOG: EQUAL  - total:  block: 87280.0 prec: 87280 diff: 0.0
GSU08RBLE: EQUAL  - total:  block: 112313.00000000001 prec: 112313 diff: 1.4551915228366852e-11
GSU08DBRO: EQUAL  - total:  block: 74565.0 prec: 74565 diff: 0.0
GSU10DBAL: EQUAL  - total:  block: 58126.0 prec: 58126 diff: 0.0
GSU10RHAC: EQUAL  - total:  block: 109456.0 prec: 109456 diff: 0.0
GSU12RHUF: EQUAL  - total:  block: 129217.99999999997 prec: 129218 diff: -2.9103830456733704e-11
GSU12DPOL: EQUAL  - total:  block: 33800.0 prec: 33800 diff: 0.0
GSU14RJOH: EQUAL  - total:  block: 127587.99999999997 prec: 127

In [152]:
column_total_check(election_cols(prec_prim_sldu_gdf), b_merge, prec_prim_sldu_gdf)

PSU02DODO: EQUAL  - total:  block: 12169.999999999998 prec: 12170 diff: -1.8189894035458565e-12
PSU02DSUB: EQUAL  - total:  block: 11162.0 prec: 11162 diff: 0.0
PSU04DWYE: EQUAL  - total:  block: 12568.0 prec: 12568 diff: 0.0
PSU06DFOG: EQUAL  - total:  block: 16867.0 prec: 16867 diff: 0.0
PSU06DGRI: EQUAL  - total:  block: 3643.0000000000005 prec: 3643 diff: 4.547473508864641e-13
PSU08DBRO: EQUAL  - total:  block: 18091.0 prec: 18091 diff: 0.0
PSU10DBAL: EQUAL  - total:  block: 16232.0 prec: 16232 diff: 0.0
PSU12DPOL: EQUAL  - total:  block: 10900.0 prec: 10900 diff: 0.0
PSU14DOTT: EQUAL  - total:  block: 13059.999999999998 prec: 13060 diff: -1.8189894035458565e-12
PSU16DBAI: EQUAL  - total:  block: 1880.0000000000002 prec: 1880 diff: 2.2737367544323206e-13
PSU16DDOU: EQUAL  - total:  block: 4389.0 prec: 4389 diff: 0.0
PSU16DLET: EQUAL  - total:  block: 23349.000000000004 prec: 23349 diff: 3.637978807091713e-12
PSU18DRAD: EQUAL  - total:  block: 23183.0 prec: 23183 diff: 0.0
PSU20DJOH

### County

In [180]:
county_sum_check(prec_gen_cong_gdf, b_merge, 'COUNTYFP', 'COUNTYFP20')
county_sum_check(prec_prim_cong_gdf, b_merge, 'COUNTYFP', 'COUNTYFP20')

county_sum_check(prec_gen_sldu_gdf, b_merge, 'COUNTYFP', 'COUNTYFP20') 
county_sum_check(prec_prim_sldu_gdf, b_merge, 'COUNTYFP', 'COUNTYFP20') 

county_sum_check(prec_gen_st_gdf, b_merge, 'COUNTYFP', 'COUNTYFP20')
county_sum_check(prec_prim_st_gdf, b_merge, 'COUNTYFP', 'COUNTYFP20') 

county_sum_check(prec_gen_sldl_gdf, b_merge, 'COUNTYFP', 'COUNTYFP20')
county_sum_check(prec_prim_sldl_gdf, b_merge, 'COUNTYFP', 'COUNTYFP20')

counties to check:  set()
County check complete!
counties to check:  set()
County check complete!
counties to check:  set()
County check complete!
counties to check:  set()
County check complete!
counties to check:  set()
County check complete!
counties to check:  set()
County check complete!
counties to check:  set()
County check complete!
counties to check:  set()
County check complete!


## Precinct

In [187]:
#precinct_sum_check(prec_gen_cong_gdf, b_merge, "PREC_GCD")
#precinct_sum_check(prec_prim_cong_gdf, b_merge, "PREC_PCD")
#precinct_sum_check(prec_gen_sldu_gdf, b_merge, "PREC_GSLDU")
precinct_sum_check(prec_prim_sldu_gdf, b_merge, "PREC_PSLDU")
precinct_sum_check(prec_gen_sldl_gdf, b_merge, "PREC_GSLDL")
precinct_sum_check(prec_prim_sldl_gdf, b_merge, "PREC_PSLDL")
precinct_sum_check(prec_gen_st_gdf, b_merge, "PREC_PST")
precinct_sum_check(prec_prim_st_gdf, b_merge, "PREC_PST")

precs to check:  set()
County check complete!
precs to check:  set()
County check complete!
precs to check:  set()
County check complete!
precs to check:  set()
County check complete!
precs to check:  set()
County check complete!


## Districts

In [196]:
def district_mismatch_report(b_merge, state, contest, prec_prim, prec_gen, prim_maup_dist, gen_maup_dist, baf_dist, dist_code):
    cdist = b_merge[['GEOID20', prec_prim, prec_gen, prim_maup_dist, gen_maup_dist, baf_dist]]
    cdistn = cdist[(cdist[prim_maup_dist]!=cdist[gen_maup_dist])&(cdist[prim_maup_dist]!='n/a')]
    print("There are", b_merge.shape[0], "blocks in ", state)
    print("\n",contest)
    print("There are ", cdistn.shape[0], "blocks where the precinct based assignment in the primary and general election files do not match.")
    wrong_dist = cdist[(b_merge[prim_maup_dist]==b_merge[gen_maup_dist])&(b_merge[prim_maup_dist]!=b_merge[baf_dist])&((b_merge[baf_dist]!=b_merge[gen_maup_dist]))]
    print("Where the maup assigned district for the primary DOES match the general, ",
          wrong_dist.shape[0], " blocks where neither assignment matches the BAF.")
    
    split_prec = cdistn[(cdistn[prec_prim].str.contains(dist_code))|(cdistn[prec_gen].str.contains(dist_code))]
    print("\nWhere the maup assigned district for the primary does not match general, ", 
          split_prec.shape[0], " blocks are split precincts.")
    mismatch_prec = cdistn[(~cdistn[prec_gen].str.contains(dist_code))&(cdistn[prec_gen]!=cdistn[prec_prim])]
    print("There are ", mismatch_prec.shape[0], 
          " blocks where the primary and general precinct assignments do not match, not including split blocks.")
    print("\nWhere precinct assignments do not match, in", mismatch_prec[mismatch_prec[prim_maup_dist]==mismatch_prec[baf_dist]].shape[0],  
          "blocks the district assignment from the primary file matches the BAF.")
    print("Where the precinct assignments do not match, in", 
          mismatch_prec[(mismatch_prec[prim_maup_dist]!=mismatch_prec[baf_dist])&(mismatch_prec[gen_maup_dist]!=mismatch_prec[baf_dist])].shape[0], 
          "blocks, neither matches the BAF.")

In [197]:
district_mismatch_report(b_merge, "Ohio","Congressional:",'PREC_PCD', 'PREC_GCD', 'PCONG_DIST', 'GCONG_DIST', 'CD116_cdg', 'CD-')

There are 276428 blocks in  Ohio

 Congressional:
There are  96 blocks where the precinct based assignment in the primary and general election files do not match.
Where the maup assigned district for the primary DOES match the general,  423  blocks where neither assignment matches the BAF.

Where the maup assigned district for the primary does not match general,  0  blocks are split precincts.
There are  96  blocks where the primary and general precinct assignments do not match, not including split blocks.

Where precinct assignments do not match, in 0 blocks the district assignment from the primary file matches the BAF.
Where the precinct assignments do not match, in 0 blocks, neither matches the BAF.


In [221]:
b_merge['SLDL18_sldlg'] = b_merge['SLDL18_sldlg'].str.slice(start = 1)
b_merge['SLDL18_sldug'] = b_merge['SLDL18_sldug'].str.slice(start = 1)

In [222]:
district_mismatch_report(b_merge, "Ohio","SLDL:",'PREC_PSLDL', 'PREC_GSLDL', 'PSLDL_DIST', 'GSLDL_DIST', 'SLDL18_sldlg', 'SLL-')

There are 276428 blocks in  Ohio

 SLDL:
There are  160 blocks where the precinct based assignment in the primary and general election files do not match.
Where the maup assigned district for the primary DOES match the general,  1238  blocks where neither assignment matches the BAF.

Where the maup assigned district for the primary does not match general,  160  blocks are split precincts.
There are  0  blocks where the primary and general precinct assignments do not match, not including split blocks.

Where precinct assignments do not match, in 0 blocks the district assignment from the primary file matches the BAF.
Where the precinct assignments do not match, in 0 blocks, neither matches the BAF.


In [223]:
district_mismatch_report(b_merge, "Ohio","SLDU:",'PREC_PSLDU', 'PREC_GSLDU', 'PSLDU_DIST', 'GSLDU_DIST', 'SLDL18_sldug', 'SLU-')

There are 276428 blocks in  Ohio

 SLDU:
There are  0 blocks where the precinct based assignment in the primary and general election files do not match.
Where the maup assigned district for the primary DOES match the general,  276333  blocks where neither assignment matches the BAF.

Where the maup assigned district for the primary does not match general,  0  blocks are split precincts.
There are  0  blocks where the primary and general precinct assignments do not match, not including split blocks.

Where precinct assignments do not match, in 0 blocks the district assignment from the primary file matches the BAF.
Where the precinct assignments do not match, in 0 blocks, neither matches the BAF.


### Vote col versus district assignment mismatch check

#### CD

In [359]:
def subset_election_cols(b_merge, elec_prefix, rm_col_list):
    subset_election_cols = list(b_merge.columns[b_merge.columns.str.contains(elec_prefix)])
    for col in rm_col_list:
        subset_election_cols.remove(col)

    return subset_election_cols

cong_gen_cols = subset_election_cols(b_merge, 'GCON', ['GCONG_DIST'])
cong_prim_cols = subset_election_cols(b_merge, 'PCON', ['PCONG_DIST'])

In [392]:
def cong_vote_mismatch_check(b_merge, dist_elec_cols_list, DIST_COL):
    
    mismatch = {}
    
    for col in dist_elec_cols_list:
        
        if str(col[4:6])==b_merge[DIST_COL][(b_merge[col]!=0)&(b_merge[DIST_COL]!='n/a')].unique().all():
            continue

        elif str(col[4:6])!=b_merge[DIST_COL][(b_merge[col]!=0)&(b_merge[DIST_COL]!='n/a')].unique().all():
            mismatch_str = col, b_merge[DIST_COL][(b_merge[col]!=0)&(b_merge[DIST_COL]!='n/a')].unique().all()
            mismatch[col] = b_merge[DIST_COL][(b_merge[col]!=0)&(b_merge[DIST_COL]!='n/a')].unique().all()
    print(DIST_COL, "check COMPLETE, mismatch below:")
    print(mismatch)
    return mismatch

    
cong_gen_cols_gen_dist = cong_vote_mismatch_check(b_merge, cong_gen_cols, 'GCONG_DIST')  
cong_gen_cols_prim_dist = cong_vote_mismatch_check(b_merge, cong_gen_cols, 'PCONG_DIST') 
cong_prim_cols_prim_dist = cong_vote_mismatch_check(b_merge, cong_prim_cols, 'PCONG_DIST') 
cong_prim_cols_gen_dist = cong_vote_mismatch_check(b_merge, cong_prim_cols, 'GCONG_DIST') 

GCONG_DIST check COMPLETE, mismatch below:
{}
PCONG_DIST check COMPLETE, mismatch below:
{'GCON03DBEA': '15', 'GCON03RRIC': '12', 'GCON11DFUD': '09', 'GCON11RGOR': '09', 'GCON12RBAL': '03', 'GCON12DSHE': '03', 'GCON12LSTE': '03', 'GCON15DNEW': '03', 'GCON15RSTI': '03', 'GCON16DGOD': '07', 'GCON16RGON': '11'}
PCONG_DIST check COMPLETE, mismatch below:
{}
GCONG_DIST check COMPLETE, mismatch below:
{'PCON03DBEA': '15', 'PCON03DHAR': '15', 'PCON09DKAP': '11', 'PCON09DROS': '11', 'PCON11DBEL': '16', 'PCON11DFUD': '16', 'PCON11DHOO': '16', 'PCON11DSHA': '16', 'PCON12DBEL': '03', 'PCON12DSHE': '03', 'PCON15DKIL': '03', 'PCON15DNEW': '03', 'PCON03RDUL': '15', 'PCON03RRIC': '15', 'PCON07RGIB': '16', 'PCON09RBAR': '11', 'PCON09RCON': '11', 'PCON09RCOR': '11', 'PCON09RWEB': '11', 'PCON11RGOR': '16', 'PCON11RSCH': '16', 'PCON11RTAY': '16', 'PCON12RBAL': '03', 'PCON12RDAY': '03', 'PCON15RHUN': '03', 'PCON15RSTI': '03', 'PCON07LLAP': '16'}


In [413]:
b_merge[['GEOID20','PCONG_DIST', 'GCONG_DIST', 'PREC_PCD', 'PREC_GCD']].loc[(b_merge['PCONG_DIST']!=b_merge['GCONG_DIST'])&
                                                                      (b_merge['PCONG_DIST']!='n/a')&
                                                                      (b_merge['PREC_PCD'].str.contains('CON-'))]

,GEOID20,PCONG_DIST,GCONG_DIST,PREC_PCD,PREC_GCD


PCD does not have a split in OH while GCD does have splits - SO we are going to trust the splits in the GCD file and reassign based on the GCD splits 
Only matters for pre-split the precincts do match (in NC there are cases where precincts do not match pre-split, source material for OH should be the same, so only differences should be caused by splits)

In [414]:
prec_gen_cong_gdf.shape

(9089, 48)

In [415]:
prec_prim_cong_gdf.shape

(9067, 81)

#### check precincts and prim and gen and dists

In [406]:
cong_prim_cols_gen_dist.keys().str.slice(start=4, stop=6).value_counts()

AttributeError: 'dict_keys' object has no attribute 'str'

In [ ]:
b_merge

In [393]:
cong_gen_cols_prim_dist

{'GCON03DBEA': '15',
 'GCON03RRIC': '12',
 'GCON11DFUD': '09',
 'GCON11RGOR': '09',
 'GCON12RBAL': '03',
 'GCON12DSHE': '03',
 'GCON12LSTE': '03',
 'GCON15DNEW': '03',
 'GCON15RSTI': '03',
 'GCON16DGOD': '07',
 'GCON16RGON': '11'}

In [399]:
list(cong_gen_cols_prim_dist.keys())

['GCON03DBEA',
 'GCON03RRIC',
 'GCON11DFUD',
 'GCON11RGOR',
 'GCON12RBAL',
 'GCON12DSHE',
 'GCON12LSTE',
 'GCON15DNEW',
 'GCON15RSTI',
 'GCON16DGOD',
 'GCON16RGON']

In [403]:
prec_gen_cong_gdf[list(cong_gen_cols_prim_dist.keys())]

,GCON03DBEA,GCON03RRIC,GCON11DFUD,GCON11RGOR,GCON12RBAL,GCON12DSHE,GCON12LSTE,GCON15DNEW,GCON15RSTI,GCON16DGOD,GCON16RGON
0,0,0,0,0,0,0,0,6,5,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9084,0,0,0,0,0,0,0,98,333,0,0
9085,6,5,0,0,0,0,0,0,0,0,0
9086,0,0,0,0,158,105,6,0,0,0,0
9087,3,10,0,0,0,0,0,0,0,0,0


In [377]:
b_merge[['GCON11RGOR', 'GCONG_DIST', 'PCONG_DIST']][(b_merge['GCON11RGOR']!=0)&(b_merge['PCONG_DIST']!='11')]

,GCON11RGOR,GCONG_DIST,PCONG_DIST
220307,1.0,11,09


In [386]:
b_merge[['PCON09DKAP', 'GCONG_DIST', 'PCONG_DIST']][(b_merge['PCON09DKAP']!=0)&(b_merge['GCONG_DIST']!='09')]

,PCON09DKAP,GCONG_DIST,PCONG_DIST
220307,0.384684,11,09


In [383]:
b_merge[b_merge.index==220307]

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,CD116_sldup,SLDU18_sldup,SLDL18_sldup,P0040001,P0050003,NAME20,ALAND20,AWATER20,geometry,P0040001-P0050003_x,maup_assignment_sldup,PREC_PSLDU,PSLDU_DIST,VAP_MOD_PSLDU,PSU02DODO,PSU02DSUB,PSU04DWYE,PSU06DFOG,PSU06DGRI,PSU08DBRO,PSU10DBAL,PSU12DPOL,PSU14DOTT,PSU16DBAI,PSU16DDOU,PSU16DLET,PSU18DRAD,PSU20DJOH,PSU22DJOH,PSU24DJAC,PSU26DSWA,PSU28DSYK,PSU30DFLE,PSU32DOBR,PSU02RGAV,PSU04RKEL,PSU04RLAN,PSU04RWON,PSU06RANT,PSU06RROB,PSU06RSEL,PSU08RBLE,PSU10RHAC,PSU12RHUF,PSU14RJOH,PSU14RUIB,PSU16RKUN,PSU18RCIR,PSU20RSCH,PSU22RBRA,PSU22RFAL,PSU22RHOV,PSU22RREY,PSU22RROM,PSU24RDOL,PSU26RACK,PSU26RREI,PSU28RDOW,PSU30RHOA,PSU32ROBR,PSU32RPOL,CD116_sldug,SLDU18_sldug,SLDL18_sldug,maup_assignment_sldug,PREC_GSLDU,GSLDU_DIST,VAP_MOD_GSLDU,GSU02RGAV,GSU02DODO,GSU04RLAN,GSU04DWYE,GSU06RANT,GSU06DFOG,GSU08RBLE,GSU08DBRO,GSU10DBAL,GSU10RHAC,GSU12RHUF,GSU12DPOL,GSU14RJOH,GSU14DOTT,GSU16RKUN,GSU16DLET,GSU18RCIR,GSU18DRAD,GSU20DJOH,GSU20RSCH,GSU22DHUN,GSU22RROM,GSU24RDOL,GSU24DJAC,GSU26RREI,GSU26DSWA,GSU28RDOW,GSU28DSYK,GSU30DFLE,GSU30RHOA,GSU32ROBR,GSU32DOBR,CD116_sldlp,SLDU18_sldlp,SLDL18_sldlp,P0040001-P0050003_y,maup_assignment_sldlp,PREC_PSLDL,PSLDL_DIST,VAP_MOD_PSLDL,PSL01DTHE,PSL03DJOH,PSL05DWIN,PSL06DROB,PSL07DSWE,PSL08DSMI,PSL09DBOY,PSL09DSTO,PSL10DBUR,PSL10DUPC,PSL11DHOW,PSL12DBRE,PSL12DROS,PSL13DSKI,PSL14DSWE,PSL15DCRO,PSL16DROM,PSL16DSMI,PSL17DMIL,PSL18DBOG,PSL19DLIG,PSL20DBRO,PSL21DLIS,PSL22DADH,PSL22DLEL,PSL23DDAY,PSL24DRUS,PSL25DJAM,PSL25DJAR,PSL25DJOS,PSL25DMAK,PSL25DNSI,PSL26DCRA,PSL27DBIT,PSL28DMIR,PSL31DKEL,PSL32DING,PSL32DWAS,PSL33DDEN,PSL33DGRA,PSL34DSYK,PSL35DGAL,PSL36DHAR,PSL36DSHA,PSL37DWEI,PSL38DCAM,PSL39DBLA,PSL39DHIC,PSL39DJON,PSL40DJAC,PSL41DBER,PSL43DCOX,PSL45DSOB,PSL46DSHE,PSL47DLAR,PSL49DWES,PSL50DSIM,PSL52DHOR,PSL53DNOV,PSL54DSHO,PSL55DSTE,PSL56DMIL,PSL57DADK,PSL58DHAG,PSL59DSTA,PSL60DTRO,PSL61DDUD,PSL62DROS,PSL63DBLA,PSL63DLAN,PSL63DPRO,PSL64DOBR,PSL65DDAR,PSL67DMOR,PSL68DMOU,PSL69DBEH,PSL70DBAR,PSL71DCAR,PSL73DMCC,PSL75DCLI,PSL76DWES,PSL77DWIL,PSL78DOWE,PSL79DRIC,PSL84DMON,PSL86DROB,PSL87DBAR,PSL88DLIE,PSL89DMIL,PSL91DDAI,PSL92DWOR,PSL94DONE,PSL96DDIP,PSL96DOLI,PSL97DSWO,PSL98DBEE,PSL99DDAN,PSL01RWET,PSL01RWIG,PSL02RJOH,PSL02RMAR,PSL03RGHA,PSL04RCUP,PSL05RGIN,PSL06RHAR,PSL06RHAW,PSL07RPAT,PSL09RRUS,PSL14RMCM,PSL15RKUS,PSL16RGRE,PSL17RHAS,PSL17RRUT,PSL18RPAC,PSL19RFRE,PSL20RBAE,PSL21RCOO,PSL22RSPE,PSL23RLAN,PSL24RMAN,PSL25RBUR,PSL27RBRI,PSL28RMON,PSL29RABR,PSL30RSEI,PSL33RHIL,PSL34RTOD,PSL35RSAR,PSL36RYOU,PSL37RBIG,PSL38RROE,PSL39RMUL,PSL40RPLU,PSL41RWHI,PSL42RSTU,PSL42RYOU,PSL43RCRE,PSL43RSMI,PSL44RMCM,PSL45RLOG,PSL46RSAL,PSL47RMER,PSL48ROEL,PSL49RHAA,PSL50RSTO,PSL51RCAR,PSL52RGRO,PSL52RWEL,PSL53RGUI,PSL53RHAL,PSL53RMUL,PSL54RZEL,PSL55RMAN,PSL56RLAC,PSL57RSTE,PSL58RSIM,PSL59RMAN,PSL60RPHI,PSL61RCAL,PSL62RLIP,PSL63RLOY,PSL64RYOD,PSL65RBLE,PSL65RDIL,PSL65RSCH,PSL66RBIR,PSL66RFRE,PSL66ROWE,PSL67RJOR,PSL68RCAR,PSL69RRAY,PSL70RKIC,PSL70RROB,PSL71RCLA,PSL71RFRA,PSL72RHOU,PSL73RBRO,PSL73RLAM,PSL74RDEA,PSL75RPAV,PSL76RGRE,PSL76RHAL,PSL77RLAR,PSL78RADA,PSL78RMIT,PSL78RSTE,PSL79RKOE,PSL80RPOW,PSL81RHOO,PSL82RRIE,PSL83RCRO,PSL84RMAN,PSL85RVIT,PSL86RRIC,PSL87RMCC,PSL88RCLI,PSL88ROLL,PSL88RTHO,PSL89RSWE,PSL90RBAL,PSL91RWIL,PSL92RJOH,PSL93RHAL,PSL93RSTE,PSL94REDW,PSL95RJON,PSL96RFER,PSL97RHOL,PSL98RHIL,PSL99RFOW,PSL96LHER,CD116_sldlg,SLDU18_sldlg,SLDL18_sldlg,P0040001-P0050003_x,maup_assignment_sldlg,PREC_GSLDL,GSLDL_DIST,VAP_MOD_GSLDL,GSL01DTHE,GSL01RWIG,GSL02DGRA,GSL02RJOH,GSL03RGHA,GSL03DJOH,GSL04RCUP,GSL05RGIN,GSL05DWIN,GSL06RHAW,GSL06DROB,GSL07RPAT,GSL07DSWE,GSL08DSMI,GSL09DBOY,GSL09RRUS,GSL10DUPC,GSL11DHOW,GSL12DBRE,GSL12RPOW,GSL13RHAR,GSL13DSKI,GSL14RMCM,GSL14DSWE,GSL15DCRO,GSL15RKUS,GSL16RGRE,GSL16DSMI,GSL17RHAS,GSL17DMIL,GSL18DBOG,GSL18RPAC,GSL19RFRE,GSL19DLIG,GSL20RBAE,GSL20DBRO,GSL21RCOO,GSL21DLIS,GSL22DLEL,GSL23DDAY,GSL23RLAN,GSL24RMAN,GSL24DRUS,GSL25RBUR,GSL25DJAR,GSL26DCRA,GSL27DBIT,GSL27RBRI,GSL28DMIR,GSL28RMON,GSL29RABR,GSL30IROL,GSL30RSEI,GSL31DKEL,GSL32DING,GSL33

In [379]:
b_merge[['PCON15DNEW', 'GCONG_DIST', 'PCONG_DIST']][(b_merge['PCON15DNEW']!=0)&(b_merge['GCONG_DIST']!='15')]

,PCON15DNEW,GCONG_DIST,PCONG_DIST
7818,0.131429,03,15
15661,0.284728,03,15
58432,0.691481,03,15
148246,0.078857,03,15
217355,0.936094,03,15
222012,0.232877,03,15
249612,0.155251,03,15


#### SL

In [373]:
sll_gen_cols = subset_election_cols(b_merge, 'GSL', ['PREC_GSLDU','GSLDU_DIST','VAP_MOD_GSLDU','PREC_GSLDL','GSLDL_DIST','VAP_MOD_GSLDL'])
sll_prim_cols = subset_election_cols(b_merge, 'PSL', ['PREC_PSLDU','PSLDU_DIST','VAP_MOD_PSLDU','PREC_PSLDL','PSLDL_DIST','VAP_MOD_PSLDL'])
slu_gen_cols = subset_election_cols(b_merge, 'GSU', [])
slu_prim_cols = subset_election_cols(b_merge, 'PSU', [])


def sl_vote_mismatch_check(b_merge, dist_elec_cols_list, DIST_COL):
    
    mismatch = []
    
    for col in dist_elec_cols_list:
        
        if str(col[3:5])==b_merge[DIST_COL][(b_merge[col]!=0)&(b_merge[DIST_COL]!='n/a')].unique().all():
            continue

        elif str(col[3:5])!=b_merge[DIST_COL][(b_merge[col]!=0)&(b_merge[DIST_COL]!='n/a')].unique().all():
            mismatch_str = col, b_merge[DIST_COL][(b_merge[col]!=0)&(b_merge[DIST_COL]!='n/a')].unique().all()
            mismatch.append(mismatch_str)       
    print(DIST_COL, "check COMPLETE, mismatch below:")
    print(set(mismatch))
    

sl_vote_mismatch_check(b_merge, sll_gen_cols, 'GSLDL_DIST') 
sl_vote_mismatch_check(b_merge, sll_gen_cols, 'PSLDL_DIST') 
sl_vote_mismatch_check(b_merge, sll_prim_cols, 'PSLDL_DIST') 
sl_vote_mismatch_check(b_merge, sll_prim_cols, 'GSLDL_DIST') 
sl_vote_mismatch_check(b_merge, slu_gen_cols, 'GSLDU_DIST') 
sl_vote_mismatch_check(b_merge, slu_gen_cols, 'PSLDU_DIST') 
sl_vote_mismatch_check(b_merge, slu_prim_cols, 'PSLDU_DIST') 
sl_vote_mismatch_check(b_merge, slu_prim_cols, 'GSLDU_DIST')

GSLDL_DIST check COMPLETE, mismatch below:
set()
PSLDL_DIST check COMPLETE, mismatch below:
{('GSL51RCAR', '53'), ('GSL26DCRA', '20'), ('GSL21RCOO', '24'), ('GSL49DWES', '50'), ('GSL40DJAC', '43'), ('GSL42RYOU', '41'), ('GSL72RHOU', '71'), ('GSL22DLEL', '25'), ('GSL23RLAN', '17'), ('GSL39DBLA', '43'), ('GSL48ROEL', '50'), ('GSL20RBAE', '17'), ('GSL67DMOR', '68'), ('GSL21DLIS', '24'), ('GSL35DGAL', '36'), ('GSL20DBRO', '17'), ('GSL23DDAY', '17'), ('GSL24RMAN', '21'), ('GSL39RMUL', '43'), ('GSL67RJOR', '68'), ('GSL19DLIG', '22'), ('GSL24DRUS', '21'), ('GSL19RFRE', '22')}
PSLDL_DIST check COMPLETE, mismatch below:
set()
GSLDL_DIST check COMPLETE, mismatch below:
{('PSL53RGUI', '51'), ('PSL19RFRE', '20'), ('PSL53DNOV', '51'), ('PSL25DJAR', '22'), ('PSL43RSMI', '40'), ('PSL25DMAK', '22'), ('PSL71DCAR', '72'), ('PSL21RCOO', '24'), ('PSL71RCLA', '72'), ('PSL17RHAS', '23'), ('PSL23RLAN', '24'), ('PSL18RPAC', '26'), ('PSL24RMAN', '21'), ('PSL17DMIL', '23'), ('PSL25DJOS', '22'), ('PSL17RRUT', '2

# Re-run maup with updated district info

## Congressional take 2

## General

In [ ]:
prec_gen_cong_gdf[list(cong_gen_cols_prim_dist.keys())]

In [66]:
bcdg = maup_pre_vote_setup(block_gdf, prec_gen_cong_gdf, 'PREC_GCD', "GCONG_DIST", "CONG_DIST")

bcdg_mod = assign_pop_to_zero_pop_prec(bcdg, 'PREC_GCD', 'P0040001-P0050003', 'VAP_MOD_GCD')

bcdg_votes = assign_votes(
    'VAP_MOD_GCD', election_cols(prec_gen_cong_gdf), prec_gen_cong_gdf, bcdg_mod
)

column_total_check(election_cols(prec_gen_cong_gdf), bcdg_votes, prec_gen_cong_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (9089, 48)
0  null assignments in maup series
VAP_MOD_GCD  added to precinct_gdf based on maup assignment
GCON01RCHA: EQUAL  - total:  block: 199559.9999999999 prec: 199560 diff: -8.731149137020111e-11
GCON01LKAH: EQUAL  - total:  block: 13691.999999999998 prec: 13692 diff: -1.8189894035458565e-12
GCON01DSCH: EQUAL  - total:  block: 172022.00000000006 prec: 172022 diff: 5.820766091346741e-11
GCON02DCAS: EQUAL  - total:  block: 146781.0 prec: 146781 diff: 0.0
GCON02RWEN: EQUAL  - total:  block: 230430.00000000003 prec: 230430 diff: 2.9103830456733704e-11
GCON03DBEA: EQUAL  - total:  block: 227420.00000000003 prec: 227420 diff: 2.9103830456733704e-11
GCON03RRIC: EQUAL  - total:  block: 93568.99999999999 prec: 93569 diff: -1.4551915228366852e-11
GCON04DFRE: EQUAL  - total:  block: 101897.0 prec: 101897 diff: 0.0
GCON04RJOR: EQUAL  - total:  block: 235875.00000000003 prec: 235875 diff: 2.9103830456733704e-11
GCON04LPER: EQUAL  - total: 

### Primary

**New Functions**

In [70]:
bcdp = maup_pre_vote_setup(block_gdf, prec_prim_cong_gdf, 'PREC_PCD', "PCONG_DIST", "CONG_DIST")

bcdp_mod = assign_pop_to_zero_pop_prec(bcdp, 'PREC_PCD', 'P0040001-P0050003', 'VAP_MOD_PCD')

bcdp_votes = assign_votes(
    'VAP_MOD_PCD', election_cols(prec_prim_cong_gdf), prec_prim_cong_gdf, bcdp_mod
)

column_total_check(election_cols(prec_prim_cong_gdf), bcdp_votes, prec_prim_cong_gdf)

block_gdf shape:  (276428, 15) 
 precinct_gdf shape:  (9067, 80)
0  null assignments in maup series
VAP_MOD_PCD  added to precinct_gdf based on maup assignment
PCON01DFOS: EQUAL  - total:  block: 17520.0 prec: 17520 diff: 0.0
PCON01DSCH: EQUAL  - total:  block: 36579.00000000001 prec: 36579 diff: 7.275957614183426e-12
PCON02DCAS: EQUAL  - total:  block: 40956.00000000001 prec: 40956 diff: 7.275957614183426e-12
PCON03DBEA: EQUAL  - total:  block: 44995.000000000015 prec: 44995 diff: 1.4551915228366852e-11
PCON03DHAR: EQUAL  - total:  block: 21057.0 prec: 21057 diff: 0.0
PCON04DFRE: EQUAL  - total:  block: 18078.0 prec: 18078 diff: 0.0
PCON04DLAR: EQUAL  - total:  block: 8944.0 prec: 8944 diff: 0.0
PCON04DSIT: EQUAL  - total:  block: 11036.999999999998 prec: 11037 diff: -1.8189894035458565e-12
PCON05DCAR: EQUAL  - total:  block: 7842.999999999999 prec: 7843 diff: -9.094947017729282e-13
PCON05DRED: EQUAL  - total:  block: 9079.0 prec: 9079 diff: 0.0
PCON05DRUB: EQUAL  - total:  block: 179